In [1]:
import simpy
from numpy import random
import pandas as pd 
import mysql.connector
import numpy as np
import math
import datetime
import time
from datetime import timedelta,date


################

def ReadProdData():
    
    st = time.time()
    dataweek = 23 # this is the week when the data was extracted
    year = 2023

    print('The week of data extraction: ',dataweek,' year ',year)

    datafilename = 'simdata/SAEventData_W'+str(dataweek)+'_'+str(year)+'.csv'

    column_names=['PN', 'SN','LocationName','Company_ID','Remarks','ParentBoard_ID','CurrentLocation','LastBaseLocation','ArrivalDate','TransactionUser_ID','Product_ID','ProductionLine']
    df_proddata = pd.read_csv(datafilename, names=column_names, delimiter=';')
        
    df_pnwc = pd.read_excel('simdata/PN-WC.xlsx') 
    df_pnwc = df_pnwc.rename(columns={'MATERIAL_CODE': 'PN', 'WORK_CENTER': 'Workcenter'})
    df_proddata = df_proddata.merge(df_pnwc,how = 'left')
    
    df_proddata['ArrivalDate']= pd.to_datetime(df_proddata['ArrivalDate'])
    df_proddata['ArrivalDay'] = [date(x.year,x.month, x.day) for x in df_proddata['ArrivalDate']]
    df_proddata['ArrivalTime'] = [60*60*x.hour+60*x.minute+x.second for x in df_proddata['ArrivalDate']]
    
 
    
    print('-> Production Execution data reading time: ',  round(time.time() - st,2), 'seconds')
        
    return df_proddata


def isNaN(num):
    return num != num

################


        
class Operator(object):
    def __init__(self, env,name,opid,competence):
        self.env = env
        self.name = name
        self.ID = opid     
        self.efficiency = 1
        self.operationexecutions = []
        self.location = None # should be one workstation
        self.status = 'idle'
        self.currentproducts = []
        self.idletime = None
        self.currentexecution = None
        self.currenttrays = []
        self.pendingexecutions = [] # list of tuples (previousexecution,batch)
        self.traycapacity = 4
        self.stopped = False
        self.inbreak = False
        self.shiftstart = 0
        self.shiftend = 0
        self.beta = 0.1
        
        
        self.performance = dict()  # key: workcnt, val: list [(PN,Perf)] where Perf: dict of key: day, val: (loc,#scans) 
        self.availability = dict() #  key: day, value: activeperiod. 
        self.pastoperations = dict() # key: day, value: list of operations performed
        self.breaks = dict() # key: day, value: list of break periods.
        self.efficiencies = dict() # key: day, value: tuple (operation,beta value)
        self.currentbreak = None
        self.shiftbreaks = []
        
        self.competence = competence
        print('name: ',self.name,' comptence ',self.competence)
    
    def checkAvailable(self,shift):  
        
        if shift.day in self.availability:
            minval = min(self.availability[shift.day][1],60*(shift.end%1440))
            maxval = max(self.availability[shift.day][0],60*(shift.start%1440))
            #print('active period',self.availability[shift.day][0],self.availability[shift.day][1])
            #print('shift day', shift.day, shift.start,shift.end)
            #print('maxval', maxval,'minval',minval)
            if minval - maxval > 1800:
                return True   
        return False
    
    def giveAvailableTime(self,shift):
        
        if shift.day in self.availability:
            minval = min(self.availability[shift.day][1],60*(shift.end%1440))
            maxval = max(self.availability[shift.day][0],60*(shift.start%1440))
            
        return max(minval - maxval,0)
    
    def setShiftTimes(self,shift):
        
        daycount = int(shift.start/1440)
        
        self.shiftstart = daycount*1440+int(max(self.availability[shift.day][0],60*(shift.start%1440))/60)
        self.shiftend = daycount*1440+int(min(self.availability[shift.day][1],60*(shift.end%1440))/60)
        
        
        self.shiftbreaks = []
        
        #print(shift.day)
        #print(self.availability[shift.day])
        #print(self.breaks)
        
        if shift.day in self.breaks:       
            for brk in self.breaks[shift.day]:  
                breakstart = daycount*1440+int(max(brk[0],60*(shift.start%1440))/60)
                breakend = daycount*1440+int(min(brk[1],60*(shift.end%1440))/60)

                if breakend - breakstart > 0:
                    self.shiftbreaks.append((breakstart,breakend))
        
        if len(self.shiftbreaks) > 0:
            self.currentbreak = self.shiftbreaks[0]
        
        
        return
    
    def resetShiftTimes(self):
        self.shiftstart = 0
        self.shiftend = 0
        
        
    def StartBreak(self):
        
        self.inbreak = True
        
        return 
    
    def FinishBreak(self,shift):
        
        for brkidx in range(len(self.shiftbreaks)):
            
            if self.shiftbreaks[brkidx] == self.currentbreak[0]:
                if brkidx < len(self.shiftbreaks)-1:
                    self.currentbreak = self.shiftbreaks[brkidx+1]
                else:
                    self.currentbreak = None
                break
                
        self.inbreak = False
     
        return 
        
   
        
#---------------------------------------------------------------------------------------------------------------         
    def proceed(self,env,execdata,ProdSystem,shift,display,makelog):
            
        myloginfo = []
        if makelog:
            myloginfo.append('Log for operator '+str(self.ID)+' time '+str(env.now))
        
        if self.location == None:
            if makelog:
                myloginfo.append('Missing location!!')
            return myloginfo
        
        if self.shiftstart > env.now:
            if makelog:
                myloginfo.append('Not started working yet in the shift')
            return myloginfo
        
        myloginfo.append('Current executions:'+str(len(self.operationexecutions)))
        for operationexecution in self.operationexecutions:
            if operationexecution.predecessor != None: 
                if not operationexecution.predecessor.completed:
                    continue  
            if operationexecution.started:
                if operationexecution.operations[0].DesignOperation.automated and operationexecution.phase == 'main': 
                    if operationexecution.readytime <= env.now:  
                        self.CompleteOperationExecution(env,operationexecution,execdata,ProdSystem,display)
            else:  
                if operationexecution.operations[0].DesignOperation.automated and operationexecution.phase == 'main':
                    
                    self.StartOperationExecution(env,operationexecution,execdata,display) #starts operationexecution
                  
               
        if self.status == 'idle':
            if self.currentbreak != None:
                if self.currentbreak[0] <= env.now:
                    if self.currentbreak[1] > env.now:
                        if not self.inbreak:
                            print('[------ Operator',self.name, ' starts ----- B R E A K ',self.currentbreak[0],'-',self.currentbreak[1],'-----]')
                            self.StartBreak()
                    else:
                        if self.inbreak:
                            self.FinishBreak(shift)
            
            if self.inbreak:
                if makelog:
                    myloginfo.append('In break, not working')
                return myloginfo
            
            for operationexecution in self.operationexecutions:

                if not operationexecution.completed: 

                    if operationexecution.predecessor != None: 
                        if not operationexecution.predecessor.completed:
                            continue   
                    if operationexecution.operations[0].DesignOperation.automated and operationexecution.phase == 'main':
                        continue

                    self.StartOperationExecution(env,operationexecution,execdata,display) #starts operationexecution
                    if self.status == 'busy':
                        return myloginfo
                    
               
            if len(self.operationexecutions) == 0:  
                if makelog:
                    myloginfo.append('Checking operation batch..')
                self.CheckOperationBatch(shift,display)
           
            if self.location == None or self.stopped:
                self.PlaceTraysBack()
                if makelog:
                    myloginfo.append('Already stopped..')
                return myloginfo
            
            if len(self.operationexecutions) == 0:
                if makelog:
                    myloginfo.append('Checking trays..')
                self.CheckTray()
                 
            if  len(self.currenttrays) == 0:
                if makelog:
                    myloginfo.append('Checking boxes..')
                loginfo = self.CheckBox(makelog)
 
                if makelog:
                    myloginfo = myloginfo+loginfo
            if len(self.currenttrays) == 0:
                if len(self.operationexecutions) == 0:
                    if sum([sum([sum([int(p.produced) for p in tr.products]) for tr in mb.trays]) for mb in self.location.out_q.boxes]) > 0:
                        trolleyboxes = [mbx for mbx in self.location.out_q.boxes]

                        transportop = self.location.workcenter.defineTransportOperation(env)

                        prods = []               
                        for mybox in trolleyboxes:
                            for mytray in mybox.trays:
                                for prod in mytray.products:
                                    prods.append(prod)
                                    prod.transportoperations.append(transportop)

                        transport_exec = OperationExecution([transportop],prods,transportop.DesignOperation.giveProcessTime(0,0),'main',True,None) 
                        transport_exec.boxes = trolleyboxes
                        transport_exec.fromlocation = self.location.out_q
                        transport_exec.tolocation = self.location.end_q
                        self.operationexecutions.insert(0,transport_exec)


                    
           
        else:
            if self.idletime <= env.now:              
                self.CompleteOperationExecution(env,self.currentexecution,execdata,ProdSystem,display)

          
        return myloginfo 
#------------------------------------------------------------------------------------------------------
    def PlaceTraysBack(self):
      
        if len(self.currenttrays) > 0:
            print('Operator ',self.name,' places back Trays ',[t.ID for t in self.currenttrays])
            
        removedtrays = []
        for mytray in self.currenttrays:
            print('Placing back of tray ',mytray.ID,' boxes in input buffer',len(self.location.in_q.boxes))
            firstprod = mytray.products[0]
            for box in self.location.in_q.boxes:
                
                if len(box.trays) < firstprod.PO.designproduct.TraysinBox:
                    print('Tray ',mytray.ID, ' placed back to the box ',box.ID,' traus in box: ',len(box.trays)+1)
                    box.trays.append(mytray)
                    mytray.Box = box
                    removedtrays.append(mytray)
                    break
                else:
                    print('Tray not put into box: Tray ',mytray.ID,'>', len(box.trays),':',firstprod.PO.designproduct.TraysinBox)
              
        for remtray in removedtrays:
            self.currenttrays.remove(remtray)
        if len(self.currenttrays) > 0:
            print('ERROR: Trays ',[t.ID for t in self.currenttrays], 'should be back to the buffer!!')
            
        return
#----------------------------------------------------------------------------------------   
    def CheckOperationBatch(self,shift,display):
 

        if len(self.currenttrays) == 0:
            return 
        
        originaldisplay = display 
        
        currenttime = env.now
        
        batchsize = 0
        for tray in self.currenttrays:
            if sum([1-int(p.produced) for p in tray.products]) > 0:
                batchsize = len(tray.products)   
                break
    
        
        
        if sum([int(op.test) for op in self.location.assignedOperations]) >0:
            if batchsize > sum([testeq.capacity for testeq in self.location.testequipments]):
                batchsize =sum([testeq.capacity for testeq in self.location.testequipments])
        
       
        if display : 
            print('Batch size: ',batchsize,' location: ',self.location.name)
                
        batch = []
        pending = []
        inserted = []
       
        if display: 
            print('########## pending executions: ',len(self.pendingexecutions))
  
        for pendingprods in self.pendingexecutions:
            if pendingprods[0] in self.location.assignedOperations:
                inserted.append(pendingprods)
                for prodx in pendingprods[1]:   
                    if len(batch) == batchsize:
                        pending.append(prodx)
                    else:
                        batch.append(prodx)
                        
        if display:  
            print('Pending-Batch: ',[p.getPNSN() for p in batch],', pending:',[p.getPNSN() for p in pending])
   
        for pendingprod in inserted:
            self.pendingexecutions.remove(pendingprod)
 

      
        if display: 
            print(currenttime,'Currenttrays optr:',self.name,':',[tray.ID for tray in self.currenttrays])
            print('Content of currenttrays optr:',self.name,':',[[p.getPNSN() for p in tray.products] for tray in self.currenttrays])


        for tray in self.currenttrays:
            if len(tray.products) == 0:
                continue
            for prod in tray.products:  
                if prod.inprogress:
                    if display: 
                        print('Prod in progress:',prod.getPNSN())
                    continue
                if self.location in prod.currentOprComb.operations[0].workstations:
                    if len(batch) == batchsize:
                        pending.append(prod)
                    else:
                        batch.append(prod)
                else:
                    print('ERROR: Operator location',self.location.name,' not is assigned ws of desopt', prod.currentOprComb.operations[0].name)
            if len(batch) == batchsize:
                break
        if display: 
            print('Final-Batch: ',[p.getPNSN() for p in batch],' batch size:',batchsize)
   
        
        if len(batch)+len(pending) == 0:
    
            #print('Óut. cap:',self.location.out_q.capacity,', used: ',len(self.location.out_q.boxes), ', Inp.used:',len(self.location.in_q.boxes),' Opt trays:',len(self.currenttrays))
            return
        
                
        if len(pending)>0:
            for pr in pending:
                pr.inprogress = True
            self.pendingexecutions.append((self.location.assignedOperations[0],pending)) # non-fitting batches for combining later
            #print('currently added pending executions: ',self.name,[pd.getPNSN() for pd in pending],'optid ',pending[0].currentOprComb.operations[0].name)
            #print('# before pending executions: ',len(self.pendingexecutions))
  
        if display: 
            print('First  executions: ',self.name,[pd.getPNSN() for pd in batch],'optid ',0,', time ',currenttime)
        for pr in batch:
            pr.inprogress = True
        fncreturn = self.defineExecutions((None,batch),0,currenttime,shift,display)
        
        if fncreturn == None:
 
            print('Overflow from shift: ',currenttime,'trays optr:',self.name,':',[(tray.ID,len(tray.products)) for tray in self.currenttrays])
            for pnd in self.pendingexecutions:
                for prod in pnd[1]:
                    print('Prod: ',prod.getPNSN(),' inprogress: ',prod.inprogress)
                    prod.inprogress = False
            for prod in batch: 
                prod.inprogress = False
            self.PlaceTraysBack()
            self.stopped = True
            return
       
        if len(pending)>0:
            if display:
                print('# after pending executions: ',len(self.pendingexecutions))
        return

#---------------------------------------------------------------------------------------------------------------
    def CheckTray(self):
         
        prodstoprocess = []
        traytaken = False
        mytraycapacity = self.traycapacity
        
        # check tray removal
        removed = []
        for mytry in self.currenttrays:  
            if len(mytry.products) == 0:
                for bx in self.location.out_q.boxes:
                    if len(bx.trays) == 0:
                        mytry.Box = bx
                        bx.trays.append(mytry)
                        removed.append(mytry)
                        break
        for tray in removed:
            self.currenttrays.remove(tray)
                
            
        for mytry in self.currenttrays:
            
            if len(mytry.products) == 0:
                for bx in self.location.out_q.boxes:
                    if len(bx.trays) == 0:
                        mytry.Box = bx
                        bx.trays.append(mytry)
                        
      
        for box in self.location.in_q.boxes:
            addedtrays = []

            
            for tray in box.trays:
                if len(self.currenttrays) < mytraycapacity: 
      
                   
                    #print('Operator',self.name, ' picks tray', tray.ID,' from box',box.ID )
                    self.currenttrays.append(tray)
                    traytaken = True
                    tray.Box = None
                    addedtrays.append(tray)
                else:
                    break
            for mytray in addedtrays:
                box.trays.remove(mytray)
    
            if len(self.currenttrays) == mytraycapacity:
                break
         
        if len(self.currenttrays) > 0:
            print('CheckTray: ',self.name,' has ',[t.ID for t in self.currenttrays],' trays')
        
             
        return 
#----------------------------------------------------------------------------------------------------------------
   
#----------------------------------------------------------------------------------------------------------------
    def CheckBox(self,writelog):
        
        fromlocation = None
        trolleycapacity = 2 
        trolleyboxes = []
        
        loginfo = []
        
        for mybox in self.location.start_q.boxes:
            if mybox.reserved:
                if writelog:
                    loginfo.append('Box'+str(mybox.ID)+ 'reserved..')
                continue
            if len(mybox.trays) == 0:
                if writelog:
                    loginfo.append('Box'+str(mybox.ID)+ ' is empty..')
                continue
            myprod = mybox.trays[0].products[0]
            
            if self.location in myprod.currentOprComb.operations[0].workstations:
                
                if len(trolleyboxes) < trolleycapacity:
                    trolleyboxes.append(mybox)
                    fromlocation = self.location.start_q     
                else:
                    break
            else:
                if writelog:
                    loginfo.append('Location of optr '+self.location.name+'not in '+str([w.name for w in myprod.currentOprComb.operations[0].workstations]))
                    
        for bx in trolleyboxes:
            bx.reserved = True

        if len(trolleyboxes) == 0:
                
            for ws in self.location.workcenter.workstations:
                if ws.name == self.location.name:
                    continue
                for mybox in ws.out_q.boxes:
                    
                    if len(mybox.trays) == 0:
                        continue
                    if len(mybox.trays[0].products) == 0:
                        continue
                        
                    myprod = mybox.trays[0].products[0]
                    
                    if len(mybox.trays) < myprod.PO.designproduct.TraysinBox:
                          
                        if sum([int(len(o.currenttrays) > 0) for o in ws.assignedOperators]) > 0:
                            continue
                        if sum([int(len(b.trays)>0) for b in ws.in_q.boxes]) > 0:
                            continue
                                
                    if myprod.currentOprComb == None:
                        continue
                                
                    if ws in myprod.currentOprComb.operations[0].workstations:
                        continue
                        
                    if self.location in myprod.currentOprComb.operations[0].workstations:
                        if len(trolleyboxes) < trolleycapacity:
                            trolleyboxes.append(mybox)
                            fromlocation = ws.out_q

                if len(trolleyboxes) > 0:
                    break  
                
        if len(trolleyboxes) > 0:
           
            transportop = self.location.workcenter.defineTransportOperation(env)
           
            prods = []               
            for mybox in trolleyboxes:
                for mytray in mybox.trays:
                    for prod in mytray.products:
                        prods.append(prod)
                        prod.transportoperations.append(transportop)
                        

            transport_exec = OperationExecution([transportop],prods,transportop.DesignOperation.giveProcessTime(0,0),'main',True,None) 
            transport_exec.fromlocation = fromlocation
            transport_exec.tolocation = self.location.in_q
            transport_exec.boxes = trolleyboxes
            self.operationexecutions.insert(0,transport_exec)
                       
        return loginfo          
#----------------------------------------------------------------------------------------------------------------
    def defineExecutions(self,exectuple,optid,starttime,shift,display):

        prev_exec = exectuple[0]
        batch = exectuple[1]
        firstprod = batch[0]
        desopt = firstprod.currentOprComb.operations[optid]
        
        proctimes = []
        st_times = []
        currenttime = starttime        
        current_exec = None
        
        
        beta = 0
        if shift.day in self.efficiencies:
            beta = self.efficiencies[shift.day][1] 
 
        st_times = [desopt.giveSetupTime() for myprod in batch]
           
         
        # determine setup and process times from sampling 
        if not desopt.parallel:
            
            proctimes = [desopt.giveProcessTime(firstprod.currentOprComb.alpha,beta) for myprod in batch]
        else:
            proctime =  desopt.giveProcessTime(firstprod.currentOprComb.alpha,beta)
            proctimes = [proctime for myprod in batch]
   

         # calculating completion time
        if prev_exec == None:
            expcomptime = currenttime
            
           
           
            
            for desgnopt in firstprod.currentOprComb.operations:
                
                precproctime = 0
                if not desgnopt.parallel:
                    precproctime += sum([1.5*desgnopt.giveSetupTime()+desgnopt.giveProcessTime(firstprod.currentOprComb.alpha,beta) for myprod in batch])
                else:
                    precproctime += 1.5*desgnopt.giveSetupTime() + desgnopt.giveProcessTime(firstprod.currentOprComb.alpha,beta)

                expcomptime+=1.25*precproctime 
                
            if expcomptime >= self.shiftend:
                
                print('->',round(currenttime,0),'Operator ',self.ID,' Overflow from shift: ',[myprod.getPNSN()  for myprod in batch],', hence returned')
                return None
          
     
        if desopt.automated: # loading phase (manual&serial)
            for prodid in range(len(batch)): 
                prod = batch[prodid]
                currenttime+= st_times[prodid]
                current_exec = OperationExecution([prod.getCombOperation(optid)],[prod],st_times[prodid],'loading',False,prev_exec)
                if display:
                    PrintOpExe(current_exec,prev_exec,currenttime)
                self.operationexecutions.append(current_exec)
                prev_exec = current_exec
                
        if desopt.parallel:
            exeproctime = 0
            
            if desopt.automated:
                currenttime+= proctimes[0]  
                exeproctime = proctimes[0]
            else:
                currenttime = starttime+1.5*st_times[0]+proctimes[0]
                exeproctime = 1.5*st_times[0]+proctimes[0]
  
            current_exec = OperationExecution([p.getCombOperation(optid) for p in batch],batch,exeproctime,'main',False,prev_exec)
 
            if display:
                PrintOpExe(current_exec,prev_exec,currenttime)
            self.operationexecutions.append(current_exec)
            prev_exec = current_exec
            
        else:
            for prodid in range(len(batch)):
                exeproctime = 0
                prod = batch[prodid]
                
                if desopt.automated:
                    currenttime+= proctimes[prodid]
                    exeproctime = proctimes[prodid]
                else:
                    currenttime+=1.5*st_times[prodid]+ proctimes[prodid] 
                    exeproctime = 1.5*st_times[prodid]+ proctimes[prodid] 
                    
                current_exec = OperationExecution([prod.getCombOperation(optid)],[prod],exeproctime,'main',False,prev_exec)      
                if display:
                    PrintOpExe(current_exec,prev_exec,currenttime)
                self.operationexecutions.append(current_exec)
                prev_exec = current_exec

        if desopt.automated: # unloading phase (manual&serial) 
            for prodid in range(len(batch)):
                prod = batch[prodid]
                currenttime+= 0.5*st_times[prodid]
                current_exec = OperationExecution([prod.getCombOperation(optid)],[prod],0.5*st_times[prodid],'unloading',False,prev_exec) 
                if display:
                    PrintOpExe(current_exec,prev_exec,currenttime)
                self.operationexecutions.append(current_exec)
                prev_exec = current_exec

                
        if desopt.name.find('Repair')>-1 or desopt.name.find('Transport')>-1:
            return prev_exec
  

        if optid < len(firstprod.currentOprComb.operations)-1:
            
            prevbatch = [p for p in batch] 
            nextdesopt = firstprod.currentOprComb.operations[optid+1]
    
            batchsize = nextdesopt.getBatchSize(firstprod.PO.designproduct,self.location)
 
            pending = []
            inserted = []
            
            for pendingprods in self.pendingexecutions:
                if pendingprods[0] == nextdesopt:
                    inserted.append(pendingprods)
                    for prodx in pendingprods[1]:
                        prevbatch.append(prodx)
          
            for pendingprod in inserted:
                self.pendingexecutions.remove(pendingprod)
      
            currentlypending = []
              
            currentlypending.append((prev_exec,prevbatch[:min(batchsize,len(prevbatch))]))
                                    
            if len(prevbatch[min(batchsize,len(prevbatch)):]) > 0:
                currentlypending.append((prev_exec,prevbatch[min(batchsize,len(prevbatch)):]))
              
            
            #print('->>> PrevBatch: ',[p.getPNSN() for p in prevbatch],'len(currentlypending)',len(currentlypending))

           
            if len(currentlypending) > 0:   
                #print('Recursion executions: ',[pd.getPNSN() for pd in currentlypending[0][1]],'optid',(optid+1),', time ',currenttime)
                self.defineExecutions(currentlypending[0],optid+1,currenttime)
                
            if len(currentlypending) == 2:
                #print('# in recursion, pending executions: ',len(self.pendingexecutions)+1)
                #print('********* Pending execution: ',[pd.getPNSN() for pd in currentlypending[1][1]],'optid',(optid+1),' op',currentlypending[1][0].operation.name)
                self.pendingexecutions.append((currentlypending[1][0].operation,currentlypending[1][1])) # non-fitting batches for combining later 
                   
        return prev_exec
                  

#----------------------------------------------------------------------------------------------------------------
    def StartOperationExecution(self,env,operationexec,execdata,display):
    
        starttime = env.now
        
        if display:
            print('->',round(starttime,2),' St_OpExe: ',self.name,',',operationexec.operations[0].DesignOperation.name,[p.getPNSN() for p in operationexec.batch],' phase: ',operationexec.phase)

        operationexec.starttime = starttime
        operationexec.readytime = starttime+operationexec.duration
        
        if operationexec.phase == 'main':
            for opt in operationexec.operations:
                opt.starttime = operationexec.starttime
             
        
        #transportation: boxes are removed from the buffer
        if operationexec.transport:
            for mybox in operationexec.boxes:
                #print('Box',mybox.ID,' gets removed from ',operationexec.fromlocation.name)
                operationexec.fromlocation.boxes.remove(mybox)
                workcnt = operationexec.batch[0].PO.designproduct.workcenter
                operationexec.fromlocation.placeBoxes(workcnt)
                mybox.location = None

                
        for prod in operationexec.batch:
   

            
            curroperation = prod.getCurrentOperation()
            if operationexec.transport:
                curroperation = prod.transportoperations[-1]
    
            if not curroperation in operationexec.operations:
                print('ÉRROR: Operation ',curroperation.DesignOperation.name, 'is not in operations of execution!!!' )

            if execdata:
                
                locname = ''
                if self.location != None:
                    locname = self.location.name
                
                newrow = {'ProductionOrder':prod.PO.ID
                          ,'PN':prod.PO.designproduct.PN,'SN':prod.SN
                          ,'LocationName':locname
                          ,'TransactionUser_ID':self.name,'Time':starttime
                          ,'Event':str(curroperation.DesignOperation.name)+"_"+operationexec.phase+"_st"}    
                prod.PO.designproduct.workcenter.currentShift.addExecution(newrow)
            
          
        if not operationexec.operations[0].DesignOperation.automated or operationexec.phase == 'loading' or operationexec.phase == 'unloading':
            self.status = 'busy'
            self.idletime = operationexec.readytime
            self.currentexecution = operationexec
            #print(round(starttime,2),'Opt',self.name, ' is busy with ',operationexec.operations[0].DesignOperation.name, 'up to ',self.idletime)
        
        operationexec.started = True
        return
    
#---------------------------------------------------------------------------------------------------------------
    def CompleteOperationExecution(self,env,operationexec,execdata,ProdSystem,display):
    
        completiontime = env.now
   
        operationexec.completiontime = completiontime
    
        if operationexec.phase == 'main':
            for opt in operationexec.operations:
                opt.completiontime = operationexec.completiontime
    
        if display:
            print(round(completiontime,2),' Comp_OpExe: ',self.name,',',operationexec.operations[0].DesignOperation.name,[p.getPNSN() for p in operationexec.batch],' phase: ',operationexec.phase)
      
        if operationexec.transport:
            print(' -->>> Transport (Optr ',self.ID, '): boxes ',[b.ID for b in operationexec.boxes],', ',operationexec.fromlocation.name,' >>>> ',operationexec.tolocation.name)
            print('    >> Transport Trays ',[[t.ID for t in b.trays] for b in operationexec.boxes])
           
            for mybox in operationexec.boxes:
                #print('Box',mybox.ID,', has moved to location: ',operationexec.tolocation.name)
                mybox.location = operationexec.tolocation
                if not mybox in operationexec.tolocation.boxes: 
                    operationexec.tolocation.boxes.append(mybox)
                            
                prods = operationexec.tolocation.getNrProducts()                    
                self.location.in_q.capacityuse.append((env.now,prods))
            curprod = operationexec.boxes[0].trays[0].products[0]
            if len(curprod.PO.products) == curprod.PO.produced:
                if operationexec.tolocation == self.location.end_q:
                    if sum([int(p.Tray.Box.location == self.location.end_q) for p in curprod.PO.products]) == len(curprod.PO.products):
                        curprod.PO.setCompleted()
                   
                        
        endbuffertransportcheck = False      
    
      
        for prod in operationexec.batch:
            
            curroperation = prod.getCurrentOperation()
            if operationexec.transport:
                curroperation = prod.transportoperations[-1]
                
                 
            if execdata:
                newrow = {'ProductionOrder':prod.PO.ID,'PN':prod.PO.designproduct.PN,'SN':prod.SN,'LocationName':self.location.name,'TransactionUser_ID':self.name,'Time':completiontime,'Event':str(curroperation.DesignOperation.name)+"_"+operationexec.phase+"_cp"}    
                prod.PO.designproduct.workcenter.currentShift.addExecution(newrow)
                
          
            # test operation
            if curroperation.DesignOperation.test: 
                    
                maxrepairs = 5
                repaireffect = len(prod.repairoperations)/maxrepairs
                if random.uniform(0, 1) > 1: #curroperation.DesignOperation.SuccessRate+repaireffect*(1-curroperation.DesignOperation.SuccessRate): 
                    prod.failreported = True
                    print('Fail case of prod: ',prod.getPNSN(), ', op: ',curroperation.DesignOperation.name)
                        
            # manual operation 
            if not curroperation.DesignOperation.automated:
                    
                if not curroperation in prod.repairoperations and not curroperation in prod.transportoperations:
                    
                    #print(prod.getPNSN(),' Op_Comb: ',[x.name for x in prod.currentOprComb.operations],', on-goingoperation: ',prod.getCurrentOperation().DesignOperation.name)
                    if prod.currentOprIndex  < len(prod.currentOprComb.operations)-1: 
                        prod.currentOprIndex+=1
                    else: 
 
                        if prod.getCombIndex() == len(prod.myroute)-1: 
                            prod.currentOprComb = None
                            prod.produced = True
                                
                            #print(prod.getPNSN(), ' is produced, current trays ',[(mytray.ID,sum([int(p.produced) for p in mytray.products])) for mytray in self.currenttrays])
                            for mytray in self.currenttrays:
                                if mytray == prod.Tray:
                                    print('Tray ',mytray.ID,':',sum([int(p.produced) for p in mytray.products]), len(mytray.products))
                                    if sum([int(p.produced) for p in mytray.products]) == len(mytray.products):
                                        for mybox in self.location.out_q.boxes:
                                            if len(mybox.trays) < prod.PO.designproduct.TraysinBox:
                                                self.currenttrays.remove(mytray)
                                                mybox.trays.append(mytray)
                                                mytray.Box = mybox
                                                print('->',round(completiontime,0),'Production complete (manual): Tray',mytray.ID,[p.getPNSN() for p in mytray.products ], ' -> box ',mytray.Box.ID, ' in ',self.location.out_q.name)
                                                #print('Operator ',self.name,' has ',len(self.currenttrays))
                                                prods = self.location.out_q.getNrProducts()
                                                self.location.out_q.capacityuse.append((env.now,prods))
                                                break
                         
                            #print('Output buffer ',self.location.out_q.name,' state: ', [[(tr.ID,sum([int(p.produced) for p in tr.products])) for tr in mb.trays] for mb in self.location.out_q.boxes])   
                                                
                            prod.PO.produced+=1 
              

                        else:  
                            prod.currentOprComb = prod.myroute[prod.getCombIndex()+1][0]
                            prod.currentOprIndex = 0
                            prod.inprogress = False
                           
                            for mytray in self.currenttrays:
                                if mytray == prod.Tray:
                                    if sum([int(prod.currentOprComb == p.currentOprComb) for p in mytray.products]) == len(mytray.products):
                                        for mybox in self.location.out_q.boxes:
                                            if len(mybox.trays) < prod.PO.designproduct.TraysinBox:
                                                self.currenttrays.remove(mytray)
                                                mybox.trays.append(mytray)
                                                mytray.Box = mybox
                                                print('->',round(completiontime,0),'WS change (manual): Tray',mytray.ID, ' -> box ',mybox.ID, ' in ',self.location.out_q.name)
                                                prods = self.location.out_q.getNrProducts()
                                                self.location.out_q.capacityuse.append((env.now,prods))
                                                break
                                                
                            outbufferfull = self.location.IsOutPutBufferFull(prod.PO.designproduct)
                            #print('Output buffer full: ',outbufferfull)
                
                            if outbufferfull:
                                trolleyboxes = [mbx for mbx in self.location.out_q.boxes]
                            
                                transportop = self.location.workcenter.defineTransportOperation(env)
           
                                prods = []               
                                for mybox in trolleyboxes:
                                    for mytray in mybox.trays:
                                        for prod in mytray.products:
                                            prods.append(prod)
                                            prod.transportoperations.append(transportop)
                                           
                                transport_exec = OperationExecution([transportop],prods,transportop.DesignOperation.giveProcessTime(0,0),'main',True,None) 
                                transport_exec.boxes = trolleyboxes
                                transport_exec.fromlocation = self.location.out_q
                                # find destination here
                                transport_exec.tolocation = prod.currentOprComb.operations[0].workstations[0].in_q
                                self.operationexecutions.insert(0,transport_exec)
                                
                                           
        
            else: # automated operation
                
                if operationexec.phase == 'unloading': 
                    if prod.currentOprIndex == len(prod.currentOprComb.operations)-1: 
                        if prod.getCombIndex() == len(prod.myroute)-1: 
                            prod.currentOprComb = None
                            prod.produced = True
                            prod.PO.produced+=1 
                            #print(prod.getPNSN(), ' is produced, tray',prod.Tray.ID,' current trays ',[(mytray.ID,sum([int(p.produced) for p in mytray.products])) for mytray in self.currenttrays])
                            
                            outbufferfull = False
                            for mytray in self.currenttrays:
                                if mytray == prod.Tray:
                                    if sum([int(p.produced) for p in mytray.products]) == len(mytray.products):
                                        for mybox in self.location.out_q.boxes:
                                            if len(mybox.trays) < prod.PO.designproduct.TraysinBox:
                                                self.currenttrays.remove(mytray)
                                                mybox.trays.append(mytray)
                                                mytray.Box = mybox
                                                print('->',round(completiontime,0),'Production complete (unloading): Tray',mytray.ID, ' -> box ',mytray.Box.ID, ' in ',self.location.out_q.name)
                                                #print('Operator ',self.name,' has ',len(self.currenttrays))
                                                outbufferfull = self.location.IsOutPutBufferFull(prod.PO.designproduct)
                                                #print('Output buffer ',self.location.out_q.name,' state: ', [[(tr.ID,sum([int(p.produced) for p in tr.products])) for tr in mb.trays] for mb in self.location.out_q.boxes])   
                                                #print('Output buffer full: ',outbufferfull)
                                                prods = self.location.out_q.getNrProducts()
                                                self.location.out_q.capacityuse.append((env.now,prods))   
                                                break

                
                            if outbufferfull:
                                trolleyboxes = [mbx for mbx in self.location.out_q.boxes]
                            
                                transportop = self.location.workcenter.defineTransportOperation(env)
           
                                prods = []               
                                for mybox in trolleyboxes:
                                    for mytray in mybox.trays:
                                        for prod in mytray.products:
                                            prods.append(prod)
                                            prod.transportoperations.append(transportop)
                                           
                                transport_exec = OperationExecution([transportop],prods,transportop.DesignOperation.giveProcessTime(0,0),'main',True,None) 
                                transport_exec.boxes = trolleyboxes
                                transport_exec.fromlocation = self.location.out_q
                                transport_exec.tolocation = self.location.end_q
                                self.operationexecutions.insert(0,transport_exec)
                                #print('Transport execution created to end buffer!!!!!!!!!!!!!!')
            
            
         
                        
                        else:
                            prod.currentOprComb = prod.myroute[prod.myroute.index(prod.currentOprComb)+1][0]
                            prod.currentOprIndex = 0
                            for mytray in self.currenttrays:
                                if mytray == prod.Tray:
                                    if sum([int(prod.currentOprComb == p.currentOprComb) for p in mytray.products]) == len(mytray.products):
                                        for mybox in self.location.out_q.boxes:
                                            if len(mybox.trays) < prod.PO.designproduct.TraysinBox:
                                                self.currenttrays.remove(mytray)
                                                mybox.trays.append(mytray)
                                                mytray.Box = mybox
                                                print('->',round(completiontime,0),'WS change (unloading): Tray',mytray.ID, ' -> box ',mybox.ID, ' in ',self.location.out_q.name)
                                                    
                                                prods = self.location.out_q.getNrProducts()
                                                self.location.out_q.capacityuse.append((env.now,prods))
                                                break
                                                 
                                        
                                

                    else:
                        prod.currentOprIndex+=1

                        
        if operationexec == self.currentexecution:
            self.currentexecution = None
            self.status = 'idle'
            
          
            
        
        operationexec.completed = True
        self.operationexecutions.remove(operationexec)
    
     
        return
#----------------------------------------------------------------------------------------------------------------     

def PrintOpExe(curr_exec,prev_exec,currenttime):
    
    if prev_exec == None:
        print('OpExe defined: ',curr_exec.operations[0].DesignOperation.name,[p.getPNSN() for p in curr_exec.batch],curr_exec.phase,round(curr_exec.duration,2),round(currenttime,2),' no pred. ')
    else:
        print('OpExe defined: ',curr_exec.operations[0].DesignOperation.name,[p.getPNSN() for p in curr_exec.batch],curr_exec.phase,round(curr_exec.duration,2),round(currenttime,2),' pred.',prev_exec.operations[0].DesignOperation.name,prev_exec.phase,[p.getPNSN() for p in prev_exec.batch])


class Machine(object):
  
    def __init__(self, env, name,speed):
        self.env = env
        self.name = name
        self.speed = speed
        self.location = None # should be one workstation

    
    def produce(self):
        while True:
            
            if self.location.in_q.items == 0:
                product = yield self.location.in_q.get()
            else:        
                product = yield self.location.in_q.get()
                
            print(f'{self.env.now:.2f} Machine {self.name} has got a part')
            
            yield env.timeout(self.speed)
            print(f'{self.env.now:.2f} Machine {self.name} finish a part next q has {len(self.location.out_q.items)} and capacit of {self.location.out_q.capacity}')

            yield self.location.out_q.items(product)
           

                


#***************************** P R O D U C T I O N       M A N A G E M E N T  ************************************
class SupplyChainManager(object):   
    def __init__(self,name):
        self.POs = []     
      
    
    def ReadProductionOrders(self,DesignMgr,ProdMangr,weekstart):
        
        df_POs = pd.read_csv('simdata/ProductionOrders.csv')
        df_POs['Date']= pd.to_datetime(df_POs['Date'])
        
        
        

        wkstart = date(weekstart.year, weekstart.month, weekstart.day)
        wkend = weekstart+ timedelta(days = 5)
        wkend = date(wkend.year,wkend.month,wkend.day)
        
        for x,r in df_POs.iterrows():
            
         
            time = 0 
            
            if time%1440 < 420:
                time = int(time/1440)*1440+420
            
            print('PO time: ',time)
            print(r)
            PO = ProductionOrder(env,'ProdOrd'+str(len(self.POs)),DesignMgr.getDesignProduct(r['PN']),r['Quantity'],time)
            self.POs.append(PO)   
            print('Productionorder of PN',PO.getPN(),' is defined')
            
            
            df_output = pd.read_csv('simdata/SystemOutput.csv')   
            df_output['Day'] = df_output['Day'].astype('string') 
            df_output['Day'] =  [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in df_output['Day'] ]
            print('>> All Output information  size: ',len(df_output))
            
            
            print(df_output.info())
         
            print(type(wkstart),type(wkend))
            sumOutput = sum(df_output[(df_output['Operation'] == '140 FT') & (df_output['Day'] >= wkstart) & ((df_output['Day'] <= wkend))]['DayOutput'])
            
            
            df_buffer = pd.read_csv('simdata/SystemBuffer.csv')            
            df_buffer['Day'] =  [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in df_buffer['Day'] ]
            print('>> All buffer information  size: ',len(df_buffer))
                    
       
            df_subbuffer = df_buffer[(df_buffer['PN'] == PO.designproduct.PN) & (df_buffer['Operation'] != '170 Customer')]
            print('>> Buffer information for PO',PO.designproduct.PN,':',len(df_subbuffer))
            print(df_subbuffer['Operation'].unique())
            
            df_items = df_subbuffer.groupby(['PN','SN'], dropna=True).sum().reset_index()
            
            
            print('Items: ',len(df_items),' sumOutput: ',sumOutput)
            
            #print(' sumOutput: ',sumOutput)
            
            for x,row in df_items.iterrows():
                PO.PNSNList.append((row['PN'],row['SN']))
                product = Product(env,PO,row['SN'],0)
                PO.products.append(product)
            
            for prod in range(len(PO.products),sumOutput):
                PO.PNSNList.append((PO.getPN(),prod))
                product = Product(env,PO,prod,0)
                
                if product.currentOprComb == None:
                    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
                PO.products.append(product)
                
            
            print('Products: ',len(PO.products))
           
                
            if len(PO.PNSNList) == 0:
                prodsn = 0
                while len(PO.PNSNList) < PO.Quantity: 
                    PO.PNSNList.append((PO.designproduct.PN,prodsn))
                    prodsn+=1
            
            PO.Quantity = len(PO.PNSNList)
            
            print('Quantity is updated: ',PO.Quantity )
          
            combstr = r['ParallelExecutions'] 
            combstr = combstr.replace('[', '')
            combstr = combstr.replace(']', '')
            print([x for x in combstr.split(",")])
            PO.parallelexecutions = [int(x) for x in combstr.split(",")]
            
            shiftlist = [x for x in ProdMangr.ProdPlan.keys()]
            sortedshifts = sorted(shiftlist, key = lambda x: x.start)
            ProdMangr.ProdPlan[sortedshifts[0]].append((PO,sortedshifts[0].start))
            
            
       
      
    
            
            
            
        return PO
    
    def GetPO(self,POID):
        
        for po in self.POs:
            if po.ID == POID:
                return po
            
        return None
       
    
    
class ProductDesignManager(object):   
    def __init__(self,name,ProdSystem):
        
        self.designproducts = dict()
        self.prodsystem = ProdSystem
        
    def ReadOperationsProducts(self,opsfile,prodsfile):
   
        df_prods = pd.read_csv('simdata/'+prodsfile)
  
        for index,row in df_prods.iterrows():
            prod = DesignProduct(env,row['PN'],self.prodsystem.getWorkCenter(row['Workcenter']),row['ItemsonTray'],row['TraysinBox'])
            self.designproducts[row['PN']] = prod
            
     
        df_ops = pd.read_csv('simdata/'+opsfile)
        df_ops = df_ops.convert_dtypes()
        
        print(df_ops)
     
        for index,row in df_ops.iterrows():
            mydesprod = self.getDesignProduct(row['PN'])
            desoperation = DesignOperation(env,row['Index'],row['Description'],row['Mean_PT'],row['StDev_PT'],row['Mean_ST'],row['StDev_ST'],mydesprod.getWorkCenter(),row['BatchSize'],row['Test'],row['SuccessRate'],row['Parallel'],row['Automated'])
            
            print(row['Index'])
            if row['Index'] == -1:
                mydesprod.repairoperation = desoperation
            else:
                mydesprod.operationsequence.append(desoperation)
                
            
        
        df_Comb = pd.read_csv('simdata/OperationCombinations.csv')
                
        for index,row in df_Comb.iterrows():
            mydesprod = self.getDesignProduct(row['PN'])
            combstr = row['Operations'] 
            combstr = combstr.replace('[', '')
            combstr = combstr.replace(']', '')
            combinationoperations = combstr.split(", ")
            
            print('Product',mydesprod.PN,' comb',combinationoperations,'op seq: ',[op.name for op in mydesprod.operationsequence])
            oplist = []
            # convert strings to objects
            for combop in combinationoperations:
                for op in mydesprod.operationsequence:
                    if op.name == combop:
                        oplist.append(op)
                        break
                OprComb = OperationCombination(row['Index'],oplist)
                mydesprod.route.append(OprComb)
                
           
                    
            for pn,desgnprod in self.designproducts.items():
                desgnprod.route = sorted(desgnprod.route, key = lambda x: x.index) # x is oprcombntn 

        return
        

    def getDesignProduct(self,PN):
        return self.designproducts[PN]
          

class ProductionManager(object):
    def __init__(self,name,ProdSystem):
        self.name = name
        self.prodsystem = ProdSystem
        self.ProdPlan = dict() # keys: Shift, values: [(PO,t)]
        self.Operators = dict() # key: Oprtr.ID, val: Operator
        
        
    def reportPlan(self):
        for shift,workplan in self.ProdPlan.items():
            for work in workplan:
                print('Shift(',work[0].designproduct.workcenter.name,'): start',shift.start,' end:',shift.end,', work: (',work[0].designproduct.PN,',',work[1],')')

        
    def createShifts(self,weekstart,workcnt,nodays):
        
        Workcnt = self.prodsystem.getWorkCenter(workcnt)
          
        timecount = 0
        for day in range(nodays):
            
            timecount+=420
            for shiftinday in range(2):

                mytype = 'Day'
                if shiftinday == 1:
                    mytype = 'Night'
                shiftday = date(weekstart.year,weekstart.month, weekstart.day)+timedelta(days=day)
                
                shift = Shift(shiftday,mytype,timecount,timecount+509,Workcnt) 
                print('Shift created: ',shift.start,shift.end,', day',shift.day,type(shift.day))
                self.ProdPlan[shift] = []
                timecount +=510 
                
        print('Nr.Shifts: ',len(self.ProdPlan))
                
                 
        return
    
    def createOperator(self,name,myid,wcname): 
        
        wc =  self.prodsystem.getWorkCenter(wcname)  
        
      
        
        if not myid in self.Operators: 
            operator = Operator(env,name,myid,[])
            self.Operators[myid] = operator
    
        if not wc in self.Operators[myid].performance:
            self.Operators[myid].performance[wc] = dict()
     
        return self.Operators[myid]
    
    def InitializeWC(self,ProdSystem):
        Workcnt = self.prodsystem.defineWorkcenter('SA')
        Workcnt.defineWorkStations(10)
        
        return 
        
        
       
    def InitializeWCO_OPT(self,df_proddata,ProdSystem):
  
    
        for wcname in df_proddata['Workcenter'].unique():

            if isNaN(wcname):
                continue
                
            if wcname.find('(') > 0:
                continue
                
                
            print('>>>>>>>>> Workcenter in the data',wcname)
    
                
            Workcnt = self.prodsystem.defineWorkcenter(wcname)
            Workcnt.defineWorkStations(10)
            
            df_wc = df_proddata[df_proddata['Workcenter'] == wcname]
            
           
    
            for oprtr in df_wc['TransactionUser_ID'].unique():
                if isNaN(oprtr):
                    continue
                    
                operator =  self.createOperator('Opr_'+str(oprtr),int(oprtr),wcname)
                print('>>>>>>>>>Operator ',oprtr,' in Workcenter',wcname)
    
                
                df_opr = df_wc[df_wc['TransactionUser_ID'] == oprtr]
            
                df_perform = df_opr.groupby(['PN','LocationName','ArrivalDay'], dropna=True)['SN','ArrivalTime'].agg(lambda x: list(x)).reset_index()
                
                for x,r in df_perform.iterrows():
                    
                    if not r['PN'] in operator.performance[Workcnt]:
                        operator.performance[Workcnt][r['PN']] = dict()
                        
       
                    if not r['ArrivalDay'] in operator.performance[Workcnt][r['PN']]:
                        operator.performance[Workcnt][r['PN']][r['ArrivalDay']] = dict()
                
                    
                    if not r['LocationName'] in operator.performance[Workcnt][r['PN']][r['ArrivalDay']]:
                        operator.performance[Workcnt][r['PN']][r['ArrivalDay']][r['LocationName']] = []
                    
                    for scanid in range(len(r['SN'])):
                        scan = (r['SN'][scanid],r['ArrivalTime'][scanid])
                        operator.performance[Workcnt][r['PN']][r['ArrivalDay']][r['LocationName']].append(scan)
                        
                    print('>>>>>>>>>Operator ',oprtr,' performed PN',r['PN'], 'Day',r['ArrivalDay'],'Loc',r['LocationName'], '#',len(r['SN']))
            

        return

################################################################################################################      
    def ReadResources(self,resfile):
  
        df_res = pd.read_csv('simdata/'+resfile)
    
        df_availability = pd.read_csv('simdata/'+'Availability.csv')     
        print('Availability info: ',len(df_availability))
 
        df_availability['Day'] = [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in df_availability['Day'] ]
    
    
        for opt in df_availability['TransactionUser_ID'].unique():
            optstr = str(opt)
            optstr = optstr[:optstr.find('.')]
          
            operator =  self.defineOperator(optstr,int(optstr),'SA',[])
            df_optr = df_availability[df_availability['TransactionUser_ID'] ==  operator.ID]
               
             
            competences = df_optr['Operation'].unique()

            for competence in competences: 
                if not competence in operator.competence:
                    operator.competence.append(competence)
            for x,rw in df_optr.iterrows():
                    
                if rw['Type'] == 'Active':
                    if rw['Day'] in operator.availability:
                        if rw['Start'] < operator.availability[rw['Day']][0]:
                            operator.availability[rw['Day']] = (rw['Start'],operator.availability[rw['Day']][1])
                        if rw['End'] > operator.availability[rw['Day']][1]:
                            operator.availability[rw['Day']] = (operator.availability[rw['Day']][0],rw['End'])
                    else:
                        operator.availability[rw['Day']] = (rw['Start'],rw['End'])
                if rw['Type'] == 'Break':
                    if not rw['Day'] in operator.breaks:
                        operator.breaks[rw['Day']] =[(rw['Start'],rw['End'])]
                    else:
                        operator.breaks[rw['Day']].append((rw['Start'],rw['End']))
                    
                if rw['Type'] == 'Efficiency':
                    operator.efficiencies[rw['Day']] = (rw['Operation'],rw['Start'])
                       
                    
                if not rw['Day'] in operator.pastoperations: 
                    operator.pastoperations[rw['Day']] = [rw['Operation']]
                else:
                    if not rw['Operation'] in operator.pastoperations[rw['Day']]:
                            operator.pastoperations[rw['Day']].append(rw['Operation'])
                                
            for k,v in operator.breaks.items():
                brklist = v
                brklist = sorted(brklist, key = lambda x: x[0])                 
                operator.breaks[k] = brklist
                    
            print('Operator: ',operator.ID)
            print('Availability info size:',len(df_optr))
            print('Competences:',operator.competence)
            #print('Past operations:',operator.pastoperations)
                    
            print('Availability:',operator.availability)
            
    
        for index,row in df_res.iterrows():
  
            if row['ResourceType'] == 'TestEquipment': 
                print(row['Competence'])
                compstr = row['Competence'] 
                compstr = compstr.replace('[', '')
                compstr = compstr.replace(']', '')
                complist = compstr.split(", ")
                
                test_equip =  self.defineTestEquipment(row['Name'],row['Workcenter'],complist,row['RecoverTime'],row['ItemsPerRun'])
        
       
            
        return     
            
 
        
    def defineOperator(self,name,myid,wcname,competence): 
        
       
        wc =  self.prodsystem.getWorkCenter(wcname)  
        operator = Operator(env,name,myid,competence)
        wc.operators.append(operator)   
        return operator 
    
    

    
    
    def defineTestEquipment(self,name,wcname,competence,recover,capacity): 
        wc =  self.prodsystem.getWorkCenter(wcname)  
        test_equip = TestEquipment(name,competence,recover,capacity)
        wc.testequipments.append(test_equip)             
        return test_equip
        
        
    def locateOperators(self,operations,wstations,shift,operators):

        
        if len(operators) == 0:
            reqcompetence = [op.name for op in operations]
            operators = [] 

            for optr in wstations[0].workcenter.operators:
                if optr.location != None: 
                    continue
                if not optr.checkAvailable(shift):
                    continue
                if sum([int(comp in optr.competence) for comp in reqcompetence]) == len(operations):
                    operators.append(optr)

            operators = sorted(operators, key = lambda x: -x.giveAvailableTime(shift))
        
        
      
        for ws in wstations:
            for optr in operators:
                if optr.location == None:
                    if len(ws.assignedOperators) < ws.operatorquota:
                        ws.assignoperator(optr)
                        optr.location = ws
                        optr.setShiftTimes(shift)
                        optr.stopped = False
         
        return
    
    
    
        
    
    def assignWorkStation(self,operation,ws):
     
        operation.workstations.append(ws)
        ws.assignedOperations.append(operation)
                
        if operation.test:              
            testeqs = operation.workcenter.findTestEquipment(operation,2)
            for testeq in testeqs:
                ws.testequipments.append(testeq)
                testeq.location = ws
                print('  >> Test equipment ',testeq.name, '-> Workstation ',ws.name)
               
        return
   
    def StartShift(self,workcnt,time,SupplyMngr,readstate):
        
        
        exclusionset = []
        exclusionset.append((5250,2025))
        #exclusionset.append((420,1896))
        #exclusionset.append((1860,1896))
        #exclusionset.append((3300,1896))
        #exclusionset.append((4740,1929))
        exclusionset.append((3810,2025))
        exclusionset.append((2370,2080))
        exclusionset.append((930,2025))
        
        
        shifts = [x for x in self.ProdPlan.keys() if x.workcenter.name == workcnt.name]
        shiftsorted = sorted(shifts, key = lambda x: x.start)
        
        shiftindx = 0 
        
        for shift in shiftsorted: 
            if shift.end <= time: 
                shiftindx +=1
                continue   
                
            if shift.start <= time:
                print('>>>> Configuring Workcenter ',workcnt.name,' in shift ', shift.day,shift.type,', [',shift.start,',',shift.end,']:')
   
                if shiftindx > 0:
                    prev_shift = shiftsorted[shiftindx-1]
            
            
                    if readstate:
                    
                        filename = 'simdata/Shift_'+str(prev_shift.start)+'_State_'+workcnt.name+'.csv'
                        df_state = pd.read_csv(filename)
                        print('Previous state file found, products: ',len(df_state))

                        currentPO = None

                        boxIDs = []
                        trayIDs = []

                        print('initial boxes: ',len(workcnt.bufferStart.boxes))

                        print('**Boxes: ',len(workcnt.allboxes),'**Trays: ',len(workcnt.alltrays))


                        for x,r in df_state.iterrows():

                            if currentPO == None:
                                currentPO = SupplyMngr.GetPO(r['PO'])

                            if currentPO.ID != r['PO']:
                                currentPO = SupplyMngr.GetPO(r['PO'])

                            inshift = False

                            for potuple in shift.POs:
                                if (potuple[0] == currentPO) & (potuple[1] == shift.start):
                                    inshift = True
                                    break

                            if not inshift: 
                                shift.POs.append((currentPO,shift.start))


                            product = Product(env,currentPO,r['SN'],r['NextOprtn'])
                            currentPO.products.append(product)


                            prodws = workcnt.getWorkStation(r['Buffer'])

                            boxloc = workcnt.bufferStart

                            if r['NextOprtn'] == -1:
                                boxloc = workcnt.bufferEnd
                                product.produced = True
                                product.currentOprComb = None



                            currentBox = None
                            if not int(r['BoxID']) in boxIDs:
                                currentBox = Box(len(workcnt.allboxes))
                                currentBox.location = boxloc
                                workcnt.allboxes.append(currentBox)
                                currentBox.ID = int(r['BoxID'])
                                boxloc.boxes.append(currentBox)
                                boxIDs.append(currentBox.ID)
                                #print('Box created: ',len(boxIDs))
                            else: 
                                for xbox in workcnt.allboxes:
                                    if xbox.ID == int(r['BoxID']):
                                        currentBox = xbox
                                        break

                            currenttray = None

                            if not int(r['TrayID']) in trayIDs:
                                currenttray = Tray(len(workcnt.alltrays))
                                workcnt.alltrays.append(currenttray)
                                currenttray.ID = int(r['TrayID']) 
                                currenttray.Box = currentBox
                                trayIDs.append(currenttray.ID)
                                currentBox.trays.append(currenttray)
                                #print('Tray created: ',len(trayIDs),)
                            else: 
                                for ttry in workcnt.alltrays:
                                    if ttry.ID == int(r['TrayID']):
                                        currenttray  = ttry
                                        break


                            currenttray.products.append(product)
                            product.Tray = currenttray


                            optcomb = ''
                            if product.currentOprComb != None:
                                optcomb = product.currentOprComb.operations[0].name
                            #print('Product',product.getPNSN(),' is in ',currentBox.location.name,' optcomb',optcomb,' tray',product.Tray.ID)


                        if currentPO!= None:
                            print('Products: ',len(currentPO.products),'Produced: ',len([p for p in currentPO.products if p.produced]))
                            print('Boxes: ',len(workcnt.allboxes),'Trays: ',len(workcnt.alltrays))

                    else:
                        print('No state reading...')
                        for box in workcnt.bufferStart.boxes:
                            box.reserved = False
                        for POtuple in prev_shift.POs:
                            if not POtuple[0].completed:
                                print('Products: ',len(POtuple[0].products),'Produced: ',len([p for p in POtuple[0].products if p.produced]))
                                print('Boxes: ',len(workcnt.allboxes),'Trays: ',len(workcnt.alltrays))
                                shift.POs.append((POtuple[0],shift.start))
                            

                # POs that started in one of the previous shifts..
                for POtuple in shift.POs:
                    print('Time: ',time,' shift start:  ',POtuple[1],' released: ',POtuple[0].released)
                    if POtuple[1] <= time:
                        ProdMgr.configureStateStartProduction(POtuple[0],workcnt,shift,exclusionset)
                      
                    
                
                
                postostart = []
                for POtuple in self.ProdPlan[shift]:
                    postostart.append(POtuple)
                                
                print('Production targets, shift.POs: ',len(shift.POs),', new POs to start: ',len(postostart)) 
                postostart = sorted(postostart, key = lambda x: x[1]) 
              
            
                # new POs starting in this shift.
                for POtuple in postostart:
                    shift.POs.append(POtuple)
                    ProdMgr.configureStateStartProduction(POtuple[0],workcnt,shift,exclusionset)
              
                workcnt.placeBoxes()
            
                if shiftindx == 0: 
                    # shift index is zero, read systembuffer file
                    
                    df_buffer = pd.read_csv('simdata/SystemBuffer.csv')
                    df_buffer['Day'] =  [datetime.datetime.strptime(x, '%Y-%m-%d').date() for x in df_buffer['Day'] ]  
                    print('All buffer information  size: ',len(df_buffer))
                    
                    for POtuple in shift.POs:
                        
                        df_subbuffer = df_buffer[(df_buffer['PN'] == POtuple[0].designproduct.PN) & (df_buffer['Day'] == shift.day)]
                        print('Buffer information for PO',POtuple[0].designproduct.PN,':',len(df_subbuffer))
                        
                        for opn in  df_subbuffer['Operation'].unique():
                            print('No products in buffer of operation: ',opn,':',len(df_subbuffer[df_subbuffer['Operation'] == opn]))
                        
                        
                        productcounter = 0
                        for x,r in df_subbuffer.iterrows():
                            if r['Operation'] == '170 Customer':
                                continue
                                
                            product = None
                            
                            snprods = [p for p in POtuple[0].products if p.SN == r['SN'] ]
                            
                            if len(snprods) > 0:
                                product = snprods[0]
                            else:
                                product = Product(env,POtuple[0],productcounter,-1)
                                POtuple[0].products.append(product)
                                

                            productcounter +=1
                            
                            for opcomb in product.myroute:
                                
                                if opcomb[0].operations[0].name == r['Operation']:
                                    product.currentOprComb = opcomb[0]
                                    
                                    print('Product',product.getPNSN(),' has next opn',opcomb[0].operations[0].name)
                                    print('Workstations assgined to desopt: ',[ws.name for ws in opcomb[0].operations[0].workstations])
                        
                                    boxfound = False  
                                    for mybox in workcnt.bufferStart.boxes:
                                        if mybox.DesignOperation != opcomb[0].operations[0]:
                                            continue
                                        trayfound = False
                                        for mytray in mybox.trays:
                                            if len(mytray.products) < POtuple[0].designproduct.ItemsonTray:
                                                mytray.products.append(product)
                                                product.Tray = mytray
                                                trayfound = True
                                                break  
                                                    
                                        if trayfound:
                                            boxfound = True
                                            break
                                        else:
                                            if len(mybox.trays) < POtuple[0].designproduct.TraysinBox:
                                                currtray = workcnt.AddTray(mybox)
                                                currtray.products.append(product)
                                                product.Tray = currtray
                                                boxfound = True
                                                break
                                                    
                                                    
                                            
                                    if not boxfound: 
                                        currentBox = workcnt.AddBox(True)
                                        currentBox.DesignOperation = opcomb[0].operations[0]
                                        currtray = workcnt.AddTray(currentBox)
                                        currtray.products.append(product)
                                        product.Tray = currtray
                           
                        POtuple[0].BoxProducts(self.prodsystem)
                        POtuple[0].Release(self.prodsystem) 
                        
                        print('bufferstart ',workcnt.bufferStart.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in workcnt.bufferStart.boxes]))  
      
       
             
                for POtuple in shift.POs:
                    if len(POtuple[0].products) == 0:
                        POtuple[0].CreateProducts(self.prodsystem)
                        if POtuple[1] <= time:
                            if not POtuple[0].released: 
                                POtuple[0].Release(self.prodsystem)              
     
                ops = []
                for optr in workcnt.operators:
                    if optr.checkAvailable(shift):
                        ops.append(optr)
                   
                print('Available operators',[o.ID for o in ops],' in shift: ',shift.day,shift.type)
                
                
                #for mxbx in workcnt.bufferStart.boxes:
                    #print('Box in start buffer: ',mxbx.ID)
                    #print([[p in p.PO.products for p in tr.products]for tr in mxbx.trays])
                
                workcnt.currentShift = shift
                
                print('shift set: ',workcnt.currentShift.start,' POs to produce: ',len(shift.POs))
                
                
                workcnt.bufferStart.boxes = sorted(workcnt.bufferStart.boxes, key = lambda x: -sum([len(tr.products) for tr in x.trays]))
                
                boxitems = []
                
                for box in workcnt.bufferStart.boxes:
                    boxitems.append((box.ID,sum([len(tr.products) for tr in box.trays])))
                    
                print(boxitems)
              
                print('bufferstart ',workcnt.bufferStart.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in workcnt.bufferStart.boxes]))  
      
                for ws in workcnt.workstations:
                    print('In_q buffer ',ws.in_q.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in ws.in_q.boxes]))  
                    print('Out_q buffer ',ws.out_q.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in ws.out_q.boxes]))  

                print('bufferend ',workcnt.bufferEnd.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in workcnt.bufferEnd.boxes]))  


                
                
                shiftindx +=1
                break
                
       
        return
    
    def CompleteShift(self,workcnt,writestate):
        
        if writestate:
            self.prodsystem.WriteSystemState(workcnt,workcnt.currentShift)
            workcnt.currentShift.ReportSystemUse(self)

        print('>> Shift (',workcnt.currentShift.start,',',workcnt.currentShift.end,') at ',workcnt.name,' completed')
    
        #for ws in workcnt.workstations:
            #print('Output buffer ',ws.out_q.name,' state: ', [[(tr.ID,sum([int(p.produced) for p in tr.products])) for tr in mb.trays] for mb in ws.out_q.boxes])   
        print('********** System state information ***********')
        print('Bufferstart ',workcnt.bufferStart.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in workcnt.bufferStart.boxes]))  
      
        for ws in workcnt.workstations:
            print('In_q buffer ',ws.in_q.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in ws.in_q.boxes]))  
            print('Out_q buffer ',ws.out_q.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in ws.out_q.boxes]))  
        
        print('Bufferend ',workcnt.bufferEnd.name,' items: ', sum([sum([len(tr.products) for tr in mb.trays]) for mb in workcnt.bufferEnd.boxes]))  
        print('********** System state information ***********')                

        
        for ws in workcnt.workstations:
            for desopt in ws.assignedOperations:
                desopt.workstations.clear()   
            ws.assignedOperations.clear()
            
            for optr in ws.assignedOperators:
                optr.PlaceTraysBack()   
                optr.location = None
            ws.assignedOperators.clear() 

            for testeq in ws.testequipments:
                testeq.location = None
            ws.testequipments.clear()
            
        for optr in workcnt.operators:  
            for opexec in optr.operationexecutions:
                for prod in opexec.batch:
                    prod.inprogress = False
            optr.operationexecutions.clear()  
            optr.pendingexecutions.clear()
            optr.currenttrays.clear()
            
        
        
        if writestate:
            for POJob in workcnt.currentShift.POs: 
                
                print('********************************************************************')
                print('********************************',writestate,'************************************')

                print('********************************************************************')


                df = workcnt.currentShift.ReportPOProduction(POJob[0])
                print('>PO summary, produced items: ',len(df), ' all products ',len(POJob[0].products))
                filename = 'simdata/Shift_'+str(workcnt.currentShift.start)+'_'+POJob[0].designproduct.PN+'.csv'
                df.to_csv(filename)
                workcnt.currentShift.productionreport[POJob[0]] = df
                filename2 = 'simdata/Shift_'+str(workcnt.currentShift.start)+'_'+workcnt.name+'_ExecutionData.csv'
                workcnt.currentShift.executiondata.to_csv(filename2)
                POJob[0].products.clear()

            for ws in workcnt.workstations:

                for box in ws.in_q.boxes:
                    for mytray in box.trays:
                        mytray.products.clear()
                    box.trays.clear()
                ws.in_q.boxes.clear()

                for box in ws.out_q.boxes:
                    for mytray in box.trays:
                        mytray.products.clear()      
                    box.trays.clear()
                ws.out_q.boxes.clear()

                
            workcnt.bufferStart.boxes.clear()
            workcnt.bufferEnd.boxes.clear()
            workcnt.allboxes.clear()     
            workcnt.alltrays.clear()
            
        else:
            
            for ws in workcnt.workstations:
   
                for box in ws.in_q.boxes:
                    for mytray in box.trays:
                        for prod in mytray.products:
                            prod.inprogess = False
                    if len(box.trays) > 0:
                        box.location = workcnt.bufferStart
                        workcnt.bufferStart.boxes.append(box)
                ws.in_q.boxes.clear()

            
                for box in ws.out_q.boxes:
            
                    if len(box.trays) == 0:
                        continue
                    
                    box.reserved = False
                    allproducedtrays = 0 
                    for mytray in box.trays:
                        if sum([int(p.produced) for p in mytray.products]) == len(mytray.products):
                            allproducedtrays+=1
                    if allproducedtrays == len(box.trays):
                        box.location = workcnt.bufferEnd
                        workcnt.bufferEnd.boxes.append(box)
                    else:
                        box.location = workcnt.bufferStart
                        workcnt.bufferStart.boxes.append(box)
 
                ws.out_q.boxes.clear()

            
        
            alltrays = dict() # key: opcomb , value: list of trays with products having next opcomb 
            for box in workcnt.bufferStart.boxes:
                for mytray in box.trays:
                    if len(mytray.products) > 0:
                        if mytray.products[0].currentOprComb == None:
                            print('Product ',mytray.products[0].getPNSN(),' in tray ',mytray.ID,' has no next operation')
                            if not None in alltrays:
                                alltrays[None] = []
                            alltrays[None].append(mytray)    
                        else:
                            nextopt = mytray.products[0].currentOprComb.operations[0]
                            if not nextopt in alltrays:
                                alltrays[nextopt] = []
                            alltrays[nextopt].append(mytray)
                box.trays.clear()
              
        
          
            for combtrays in alltrays.items():  

                boxcapacity = 1
                
                if combtrays[0]!=None: 
                    
                    print('Next opt: ', combtrays[0].name,' items: ',sum([len(t.products) for t in combtrays[1]]))
                    boxcapacity = combtrays[1][0].products[0].PO.designproduct.TraysinBox

                
                
                
                for box in workcnt.bufferStart.boxes:
                    
                    if len(box.trays) == 0:  
                        while (len(box.trays) < boxcapacity) & (len(combtrays[1]) > 0):
                            firsttray = combtrays[1][0]
                            firsttray.Box = box
                            box.trays.append(firsttray)
                            combtrays[1].remove(firsttray)
                        
                    if len(combtrays[1]) == 0:
                        if combtrays[0]!=None: 
                            print('All trays of ',combtrays[0].name,' are placed!!')
                        else:
                            print('All trays of none operation are placed!!') 
                        break
       
        
        workcnt.currentShift = None
        
        return

  


    def configureStateStartProduction(self,PO,workcnt,shift,exclset):
  
        
        optfound = 1
        
        while optfound > 0: 
            optfound = 0
            for opcombidx in range(len(PO.designproduct.route)): 
                comb = PO.designproduct.route[opcombidx]

                wslist = []

                operator = workcnt.WorkedOperators(shift,comb.operations[0].name)
                
               
                if operator == None:
                    continue
                    
                if (shift.start,operator.ID) in exclset and comb.operations[0].name == '150 SA':
                    continue


                curr_ws = comb.operations[0].workcenter.findWorkStation()

                if curr_ws == None:
                    continue
                else:
                    optfound+=1
                    for desopt in comb.operations:            
                        self.assignWorkStation(desopt,curr_ws)  
                        if not curr_ws in wslist:
                            wslist.append(curr_ws) 

                    self.locateOperators(comb.operations,wslist,shift,[operator])
                    print(' ---> Operation ',[o.name for o in comb.operations],' @Workstation:  ',curr_ws.name,' Optr: ',operator.ID,', Availability: [',operator.shiftstart,operator.shiftend,']')
                  

        return

    
     

#***************************** P R O D U C T I O N       M A N A G E M E N T  ************************************


#***************************** P R O D U C T I O N       S Y S T E M ************************************

class ProductionSystem(object): 
    def __init__(self,name):
        self.name = name
        self.workcenters = []
        
        
    
    def defineWorkcenter(self,name):
        print('Defining a workcenter name',name)
        Workcenter = WorkCenter(env,name) 
        self.workcenters.append(Workcenter)
        return Workcenter
        
    def getWorkCenter(self,name):
        for wc in self.workcenters:
            if wc.name == name:
                return wc
        return None
    
  
    
    def report(self):   
        for workcnt in self.workcenters:
            workcnt.report()
            
        return
    
    
    def WriteSystemState(self,wcnt,shift):
    
    
        streportcolumns=['PN','SN','PO','NextOprtn','TrayID','BoxID','Buffer']
        
        df_state = pd.DataFrame(columns=streportcolumns)
        
        stPNSNList = [] 
        stTrayIDList = [] 
        stBoxIDList = [] 
        stBufferList = []
        
        
        for POJob in wcnt.currentShift.POs:
            for prod in POJob[0].products:
                prod.reported = False
            
        
        for box in wcnt.bufferStart.boxes:
            for mytray in box.trays:
                for prod in mytray.products: 
                    pdinfo = {'PN':prod.PO.designproduct.PN,'SN':prod.SN,'PO':prod.PO.ID,'NextOprtn':prod.currentOprComb.index,'TrayID':mytray.ID,'BoxID':box.ID,'Buffer':wcnt.bufferStart.name}
                    df_state = df_state.append(pdinfo, ignore_index=True)
                    prod.reported = True

        for box in wcnt.bufferEnd.boxes:
            for mytray in box.trays:
                for prod in mytray.products: 
                    pdinfo = {'PN':prod.PO.designproduct.PN,'SN':prod.SN,'PO':prod.PO.ID,'NextOprtn':-1,'TrayID':mytray.ID,'BoxID':box.ID,'Buffer':wcnt.bufferEnd.name}
                    df_state = df_state.append(pdinfo, ignore_index=True)
                    prod.reported = True

        
        for ws in wcnt.workstations:
            for box in ws.in_q.boxes:
                for mytray in box.trays:
                    for prod in mytray.products: 
                        
                        prod.reported = True
                        if prod.produced: 
                            print('WARNING: Produced prod',prod.getPNSN(), ' is in tray', mytray.ID,' box ',box.ID,' buffer',ws.in_q.name)
                        
                        pdinfo = {'PN':prod.PO.designproduct.PN,'SN':prod.SN,'PO':prod.PO.ID,'NextOprtn':-1 if prod.produced else prod.currentOprComb.index ,'TrayID':mytray.ID,'BoxID':box.ID,'Buffer':ws.in_q.name}
                        df_state = df_state.append(pdinfo, ignore_index=True)
            for box in ws.out_q.boxes:
                for mytray in box.trays:
                    for prod in mytray.products: 
                        pdinfo = {'PN':prod.PO.designproduct.PN,'SN':prod.SN,'PO':prod.PO.ID,'NextOprtn':-1 if prod.produced else prod.currentOprComb.index,'TrayID':mytray.ID,'BoxID':box.ID,'Buffer':ws.out_q.name}
                        df_state = df_state.append(pdinfo, ignore_index=True)
                        prod.reported = True
         
        for POJob in wcnt.currentShift.POs:
            for prod in POJob[0].products:
                if not prod.reported:
                    print('ERROR: prod',prod.getPNSN(), ' NOT REPORTED!!, tray of product: ',prod.Tray.ID)
                        
       
        filename = 'simdata/Shift_'+str(shift.start)+'_State_'+wcnt.name+'.csv'
        df_state.to_csv(filename)
        
        return
                    
    
class WorkCenter(object):  
    
    def __init__(self,env,name):
        self.env = env
        self.name = name
        self.workstations = []
        self.operators = []   
        self.testequipments = []
        self.currentShift = None
        self.bufferStart = Buffer(env,'Buffer_Start',10000000,False,True,False)
        self.bufferEnd =  Buffer(env,'Buffer_End',10000000,False,False,True) 
        self.transportoperation =  DesignOperation(env,0,'transport_op',0,0,0,0,self,10000,False,1.00,True,False)
        self.alltrays = []
        self.allboxes = []
        
        
    def AddBox(self,startbuffer):
 
        currentBox = Box(len(self.allboxes)) #id setting must be improved
   
        if startbuffer:         
            currentBox.location = self.bufferStart
            self.bufferStart.boxes.append(currentBox)
           
        else:
            currentBox.location = self.bufferEnd  
            self.bufferEnd.boxes.append(currentBox)
         
        currentBox.ID = self.giveBoxID()
        self.allboxes.append(currentBox)
        
        return currentBox
    
    def AddTray(self,mybox):
        
        currenttray = Tray(len(self.alltrays))
        self.alltrays.append(currenttray)
        currenttray.ID = int(len(self.alltrays)) 
        currenttray.Box = mybox
        mybox.trays.append(currenttray)
        
        return currenttray
    
        
    def WorkedOperators(self,shift,desoptname):
        operator = None
        for optr in self.operators:
            #print('Checking Operator ',optr.name, 'worked in shift', shift.day,shift.type,' for opr ',desoptname)
            if optr.location != None:
                continue
            if shift.day in optr.pastoperations:
                #print('Operator ',optr.name, 'worked in shift', shift.day,shift.type,' for opr ',desoptname,optr.pastoperations[shift.day])
   
                if desoptname in optr.pastoperations[shift.day]:
                    if optr.checkAvailable(shift):
                        #print('-> Operator ',optr.name, 'worked in shift', shift.day,shift.type,' for opr ',desoptname)
                        operator = optr
                        break
                        
        return operator
                            
        
        
    def placeBoxes(self):
        
        for workst in self.workstations:
            while workst.out_q.capacity > len(workst.out_q.boxes):
                currentBox = Box(len(self.allboxes)) #id setting must be improved
                currentBox.location = workst.out_q
                workst.out_q.boxes.append(currentBox)
                currentBox.ID = self.giveBoxID()
                self.allboxes.append(currentBox)
                          
    def giveBoxID(self):
        
        
        self.allboxes  = sorted(self.allboxes, key = lambda x: x.ID)
        
        boxid = 1
        
        for mbox in self.allboxes:
            if mbox.ID == boxid:
                boxid+=1
            if mbox.ID > boxid:
                return boxid
            
        return boxid
        

    def defineTransportOperation(self,env):
        return Operation(env,self.transportoperation)


    def defineWorkStations(self,no):
        for ws in range(no):
            ws = WorkStation(env, str(self.name)+'_WS_'+str(ws+1),self.bufferStart,self.bufferEnd,self)
            self.workstations.append(ws)
        
        print('Workcenter',self.name,' has ',len(self.workstations),' workstations')
        return
    
    def report(self):
        print('Workcnt: ',self.name, ' has '+str(len(self.workstations))+' workstations, '+str(len(self.testequipments))+' testequipments, '+str(len(self.operators))+' operators')
        return  
    
    def findWorkStation(self):
        
        for ws in self.workstations:
            #print('Ws finding',self.name,' ops: ',len(ws.assignedOperations) ,' quota: ',ws.operationquota)
            if len(ws.assignedOperations) < ws.operationquota:
                return ws
        
        return None
    
    def getWorkStation(self,bfname):
        
        for ws in self.workstations:
            if bfname == ws.in_q.name:
                return ws
            if bfname == ws.out_q.name:
                return ws
            
        return None
 
    
    def findTestEquipment(self,operation,nrequipments):
        
        equipments = [] 
        for testeqp in self.testequipments:
            if operation.name in testeqp.competence: 
                if testeqp.status == 'Working' and testeqp.location == None:
                    equipments.append(testeqp)
            if len(equipments) == nrequipments:
                break
                    
        return equipments
            
    
    def getRepairStation(self):
        
        for ws in self.workstations:
            if ws.name.find("RepairStation") > -1:
                return ws
        return None
        
        

class WorkStation(object):  
    def __init__(self,env,name,start_q,end_q,workcnt):
        self.env = env
        self.name = name
        self.start_q = start_q
        self.end_q = end_q
        self.assignedOperators = [] 
        self.testequipments = []
        self.assignedOperations = []     
        self.in_q = Buffer(env,self.name+'_In_Buffer',2,False,False,False) 
        self.out_q = Buffer(env,self.name+'_Out_Buffer',2,True,False,False) 
        self.operatorquota = 1
        self.operationquota = 1
        self.workcenter = workcnt
        
    def IsOutPutBufferFull(self,DesgnProd):
 
        if len(self.out_q.boxes) < self.out_q.capacity:
            return False
   
        
        for box in self.out_q.boxes: 
            if len(box.trays) < DesgnProd.TraysinBox:
                return False
 
        return True

    def AddBox(self,inbuffer):
 
        currentBox = Box(len(self.workcenter.allboxes)) #id setting must be improved
   
        if inbuffer:         
            currentBox.location = self.in_q
            if self.in_q.capacity < len(self.in_q.boxes)+1:
                self.in_q.capacity+=1   
            self.in_q.boxes.append(currentBox)
           
        else:
            currentBox.location = self.out_q
            if self.out_q.capacity < len(self.out_q.boxes)+1:
                self.out_q.capacity+=1       
            self.out_q.boxes.append(currentBox)
         
        currentBox.ID = self.workcenter.giveBoxID()
        self.workcenter.allboxes.append(currentBox)
        
        return currentBox
    
    
    def AddTray(self,mybox):
        
        currenttray = Tray(len(self.workcenter.alltrays))
        self.workcenter.alltrays.append(currenttray)
        currenttray.ID = int(len(self.workcenter.alltrays)) 
        currenttray.Box = mybox
        mybox.trays.append(currenttray)
        
        return currenttray
            
    
    def assignoperator(self,operator):
        operator.location = self
        self.assignedOperators.append(operator)

    
        
class Buffer(object):   
    def __init__(self,env,name,capacity,out,start,end):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.items = []
        self.boxes = []
        self.start = start
        self.out = out
        self.end = end
        self.capacityuse = [] # list of pairs (time,capacityuse)
        
        self.capacityuse.append((env.now,0))
        

    def placeBoxes(self,workcnt):
     
        if self.capacity > 1000:
            return
        
        while self.capacity > len(self.boxes):
            currentBox = Box(len(workcnt.allboxes)) #id setting must be improved
            currentBox.location = self
            self.boxes.append(currentBox)
            currentBox.ID = workcnt.giveBoxID()
            workcnt.allboxes.append(currentBox)
            
        return 
    
    def getNrProducts(self):     
        
        nrprods = 0
       
        for box in self.boxes:
            for tray in box.trays:
                nrprods+=len(tray.products)

        return nrprods
        
    def Items(self):
        return self.items
        
  
        
class TestEquipment(object):
    def __init__(self,name,competence,recover,capacity):
        self.name = name
        self.competence = competence
        self.location = None
        self.status = 'Working'
        self.recovertime = recover
        self.inrepair = 0
        self.capacity = capacity
    
    
class Tray(object):
    def __init__(self,myid): 
        self.ID = myid
        self.Box = None
        self.products = []

class Box(object):
    def __init__(self,myid): 
        self.ID = myid
        self.trays = []
        self.DesignOperation = None
        self.location = None # buffer
        self.reserved = False


#***************************** P R O D U C T I O N       S Y S T E M ************************************


#***************************** P R O D U C T I O N       D E S I G N ************************************
class DesignProduct(object):
    def __init__(self, env,PN,WC,ItsTray,TrsBox):
        self.env = env
        self.PN = PN  
        self.operationsequence = []
        self.route = [] # list of operation groups
        self.workcenter = WC      
        self.predecessors = []
        self.repairoperation = None
        self.ItemsonTray = ItsTray
        self.TraysinBox = TrsBox
        
    def getRoute(self):
        return self.route
    
    
    def getWorkCenter(self):
        return self.workcenter
            
    def getBOM(self,level):

        strspc = ''
        if level == 0:
            print('***** Product BOM *****')
        fnlspc = strspc
        for it in range(level):
            fnlspc+= strspc
        
        routestr = ''
        for comb in self.route:
            print('ops',len(comb.operations))
            for op in comb.operations:
                if routestr != '':
                    routestr+=','+op.name
                else:
                    routestr+=op.name
        print(fnlspc,'->PN',self.PN,', Wc-route: '+routestr)
        
        for pred in self.predecessors:
            pred.getBOM(level+1)
        if level == 0:
            print('***** Product BOM *****')
            
        return
        

        

class DesignOperation(object):  
    def __init__(self,env,ind,name,meanproctime,stdevproctime,mstime,stdstime,workcnt,batch,test,srate,parallel,automated):
        self.env = env
        self.name = name
        self.index = ind
        self.test = test
        self.meanproctime = meanproctime
        self.stdevproctime = stdevproctime 
        self.meansetuptime = mstime
        self.stdevsetuptime = stdstime 
        self.batchsize = batch
        self.workcenter = workcnt   
        self.workstations = []  
        self.SuccessRate = srate
        self.parallel = parallel 
        self.automated = automated
        self.combinationindex = -1
        
    def giveProcessTime(self,alpha,beta):
        proctime = max(0.1,math.log(random.lognormal(mean=10*self.meanproctime,sigma=10*self.stdevproctime,size = 1)[0])/10)

        return (proctime*(1+alpha))*(1+beta)
        
    def giveSetupTime(self):
        return  min(max(0.01,math.log(random.lognormal(mean=10*self.meansetuptime,sigma=10*self.stdevsetuptime,size = 1)[0])/10),0.1)
        
    def getBatchSize(self,DesProd,Location):
        
        batchsize = 0   
        if self.test: 
            for testeq in Location.testequipments:
                batchsize+=testeq.capacity
        else: 
            batchsize = DesProd.ItemsonTray
                
        return batchsize 
        
class OperationCombination(object):  
    def __init__(self,index,operations):
        self.index = index
        self.operations = operations
        
        if self.operations[0].name == '140 FT':
            self.alpha = 0.1725
        if self.operations[0].name == '150 SA':
            self.alpha = 0.3
    
    
#***************************** P R O D U C T I O N       D E S I G N ************************************


#***************************** P R O D U C T I O N       E X E C U T I O N ************************************




class ProductionOrder(object):
    def __init__(self,env,name,designprod,Quantity,time):
        self.env = env
        self.ID = name
        self.designproduct = designprod
        self.products = []
        self.releasetime = time
        self.produced = 0
        self.released = False
        self.completed = False
        self.DesiredThroughput = None
        self.parallelexecutions = []
        self.Quantity = Quantity
        self.PNSNList = []
        
        
    def CreateProducts(self,ProdSystem):
        
        workcnt = self.designproduct.workcenter
        
        currentBox = Box(len(workcnt.allboxes))
        workcnt.allboxes.append(currentBox)
        currentBox.ID = workcnt.giveBoxID()
        currentTray = Tray(len(workcnt.alltrays))
        workcnt.alltrays.append(currentTray)
        currentTray.ID = len(workcnt.alltrays)
        
        
        for pnsn in self.PNSNList:
            product = Product(env,self,pnsn[1],0)
            
            currentTray.products.append(product)
            product.Tray = currentTray
        
        
            if len(currentTray.products) == self.designproduct.ItemsonTray:
                currentBox.trays.append(currentTray)
                currentTray.Box = currentBox
                currentTray = Tray(len(workcnt.alltrays))
                workcnt.alltrays.append(currentTray)
                currentTray.ID = len(workcnt.alltrays)
        
  
            if len(currentBox.trays) == self.designproduct.TraysinBox:
                currentBox = Box(len(workcnt.allboxes))
                workcnt.allboxes.append(currentBox)
                currentBox.ID = len(workcnt.allboxes)

            #ItemsonTray
            
          
            self.products.append(product)
       
        print('PO Products: ',len(self.products))
       
        return
    
    def BoxProducts(self,ProdSystem):
        
        workcnt = self.designproduct.workcenter
        
        currentBox = Box(len(workcnt.allboxes))
        workcnt.allboxes.append(currentBox)
        currentBox.ID = len(workcnt.allboxes)
        currentTray = Tray(len(workcnt.alltrays))
        workcnt.alltrays.append(currentTray)
        currentTray.ID = len(workcnt.alltrays)
        
        
        for product  in self.products:
            
            if product.Tray != None: 
                continue
            
            currentTray.products.append(product)
            product.Tray = currentTray
        
        
            if len(currentTray.products) == self.designproduct.ItemsonTray:
                currentBox.trays.append(currentTray)
                currentTray.Box = currentBox
                currentTray = Tray(len(workcnt.alltrays))
                workcnt.alltrays.append(currentTray)
                currentTray.ID = len(workcnt.alltrays)
        
  
            if len(currentBox.trays) == self.designproduct.TraysinBox:
                currentBox = Box(len(workcnt.allboxes))
                workcnt.allboxes.append(currentBox)
                currentBox.ID = len(workcnt.allboxes)

        if not currentTray in currentBox.trays:
            currentBox.trays.append(currentTray)
            currentTray.Box = currentBox
           

        return
    
     
    
    def setCompleted(self):
        
        self.completed = True
        
    
        #mydf = self.designproduct.workcenter.currentShift.productionreport
        #self.designproduct.workcenter.currentShift.productionreport = pd.concat([mydf, self.ReportPOProduction()])
  
        for optcomb in self.designproduct.route:
            # release workstation-operation assignments
            print('releasing assignments for operation',[desopt.name for desopt in optcomb.operations])
            for desopt in optcomb.operations:
                for ws in desopt.workstations:
                    ws.assignedOperations.remove(desopt)
                    for optr in ws.assignedOperators:
                        optr.location = None
                    ws.assignedOperators.clear()    
            
                desopt.workstations.clear()
           
        
        print('Production Order ',self.getPN(),' of Q = ',len(self.products), 'completed')
        
        return
        

    
    def Release(self,ProdSystem):
        
        allprodinboxes = 0
        nrboxes = 0
        nrtrays = 0
        for mybox in self.designproduct.workcenter.allboxes:
            if len(mybox.trays) == 0:
                continue
            #print('Check:',mybox.trays[0].products[0].PO.ID,': ',self.ID)
            
            if mybox.location != None:
                continue
            
            if mybox.trays[0].products[0].PO.ID == self.ID:
                allprodinboxes+=sum([len(mytray.products) for mytray in mybox.trays])
                nrboxes+=1
                nrtrays+=len(mybox.trays)
                self.designproduct.workcenter.bufferStart.boxes.append(mybox)
                mybox.location = self.designproduct.workcenter.bufferStart
                #print('Laocation of box',mybox.ID,' is ',mybox.location.name)
        print('PO ', self.ID, 'release: ',nrboxes,' boxes, ',nrtrays,' trays to start Buffer!')
            
      
        self.released = True
          
        
    def getPN(self):
        return self.designproduct.PN
    
class OperationExecution(object):    
    def __init__(self,optns,batch,proctime,phase,trsprt,predcssr):
  
        self.batch = batch
        self.operations =  optns 
        self.duration = proctime
        self.phase = phase # loading (automated),main,unloading(automated)
        self.transport = trsprt
        self.fromlocation = None
        self.tolocation = None
        
        self.completed = False
        self.started = False  
        self.readytime = None
        self.starttime = None
        self.completiontime = None
        
        if self.phase == 'main':
            for opt in self.operations:
                opt.processtime = self.duration
        if self.phase == 'loading':
            for opt in self.operations:
                opt.setuptime = self.duration
 
        # process-perspective
        for p in self.batch:
            p.operationexecutions.append(self)
            
        self.predecessor = predcssr
        
        if predcssr != None:
            predcssr.successor = self         
        self.successor = None 
        
        self.boxes = []
       
    
        
    
                
class Product(object):   
    def __init__(self, env,PO,SN,combindex):
        self.env = env
        self.PO = PO
        self.SN = SN
   
        self.currentOprCombIndex = 0 
        self.currentOprIndex = 0
        
        self.myroute = [] # list of pairs (oprcomb,operations)
        self.inprogress = False
        self.produced = False
        self.failreported = False
        self.failed = False
        self.repaired = False
        self.repairoperations = []
        self.transportoperations = []
        self.operationtostart = 0
        self.Tray = None
      
        self.operationexecutions = []
        self.reported = False
        
    
        for oprcomb in PO.designproduct.route:
            self.myroute.append((oprcomb,[Operation(env,desop) for desop in oprcomb.operations]))
         
        if combindex >= 0:
            self.currentOprComb = PO.designproduct.route[combindex]
        else:
            self.currentOprComb = None
        
     
    def getCurrentOperation(self):
        for oprtuple in self.myroute:
            if oprtuple[0] == self.currentOprComb:
                return oprtuple[1][self.currentOprIndex]
            
    def getCombOperation(self,opidx):
        for oprtuple in self.myroute:
            if oprtuple[0] == self.currentOprComb:
                return oprtuple[1][opidx]
        
    def getCombIndex(self):
        for indx in range(len(self.myroute)):
            if self.myroute[indx][0] == self.currentOprComb:
                return indx
        
    
    def getPN(self):
        return self.PO.designproduct.PN
    
    def getRoute(self):
        return self.PO.designproduct.getRoute()
        
    def getPNSN(self):
        return self.getPN()+"~"+str(self.SN)



class Operation(object):  
    def __init__(self,env,DesOperation):
        self.env = env
        self.DesignOperation = DesOperation
  
        self.starttime = None
        self.inpreparation = False
        self.inpogress = False
        self.completiontime = None
        self.processtime = None
        self.setuptime = None
        self.UserID = None
        self.location = None
        self.myexecutions = []

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
        
class Shift(object):  
    def __init__(self,day,mytype,start,end,workcnt):
        self.day = day
        self.type = mytype
        self.start = start
        self.end = end
        self.workcenter = workcnt
        self.productionreport = dict() # key: PO, value: Dataframe of execution data
        self.executiondata = pd.DataFrame(columns=['ProductionOrder','PN','SN','LocationName','TransactionUser_ID','Time','Event'])  # MFi check 5-4-23
        # newrow = {'ProductionOrder':'','PN':'','SN':1,'LocationName':'','Location_ID':'','TransactionUser_ID':0,'ArrivalDate':0}    
        self.POs = []
     
    
  
    
    def addExecution(self,row):
        self.executiondata = self.executiondata.append(row,ignore_index=True)
        
    def ReportSystemUse(self,ProdMgr):
 
        reportcolumns=['Buffer','Time','CapacityUse']
        
        df = pd.DataFrame(columns=reportcolumns)
        
        
        for wcnt in ProdMgr.prodsystem.workcenters:
            for ws in wcnt.workstations:
                
                for capuse in ws.in_q.capacityuse:
                    useitem = {'Buffer':ws.in_q.name,'Time':capuse[0],'CapacityUse':capuse[1]}
                    df = df.append(useitem, ignore_index=True)
                for capuse in ws.out_q.capacityuse:
                    useitem = {'Buffer':ws.out_q.name,'Time':capuse[0],'CapacityUse':capuse[1]}
                    df = df.append(useitem, ignore_index=True)

        df = df.sort_values(by=['Time'],ascending = True).reset_index()
        
        
        #fig, axs = plt.subplots(nrows=len(df['Buffer'].unique()))
        
        bufidx = 0
        for buffer in df['Buffer'].unique():
            df_sub = df[df['Buffer'] == buffer]        
            #sns.relplot( ax=axs[bufidx],x='Time', y='CapacityUse',data=df_sub,sizes=(40, 300), alpha=.5, palette="muted", height=9).set(title='Shift '+str(self.start)+': Buffer levels of '+buffer)
            bufidx+=1
            
                    
        filename = 'simdata/Shift_'+str(self.start)+'_CapacityUse.csv'
        df.to_csv(filename)  
        
        return
              
            
        
        
    def ReportPOProduction(self,PO):
    
        reportcolumns=['PN-SN']


        PNSNList = [] 
        DFOpStList = []
        DFOpProcList = []
        DFOpCmpList = []
        RepairsList = []

        for oprcomb in PO.designproduct.route:
            for operation in oprcomb.operations:
                DFOpProcList.append([])
                reportcolumns.append(operation.name+"_pt")
                DFOpStList.append([])
                reportcolumns.append(operation.name+"_st")
                DFOpCmpList.append([])
                reportcolumns.append(operation.name+"_c")
        reportcolumns.append("Repairs")

    
        for prod in PO.products:
 
            if not prod.produced:
                #print(prod.getPNSN(),', Not produced!!')
                continue
                
            PNSNList.append(prod.getPNSN())

            opindx = 0
            
          
            for combtuple in prod.myroute: 
               
               
                for oprtn in combtuple[1]: 
                    if oprtn.processtime == None: 
                        DFOpProcList[opindx].append(0) 
                        DFOpStList[opindx].append(0)
                        DFOpCmpList[opindx].append(0)
                    else:
                        DFOpProcList[opindx].append(round(oprtn.processtime,2))
                        DFOpStList[opindx].append(round(oprtn.starttime,2))
                        DFOpCmpList[opindx].append(round(oprtn.completiontime,2))
                    opindx+=1
                    
            rep_str = ''
            for repair in prod.repairoperations:
                if rep_str !='':
                    rep_str +='|'
                rep_str+= 'pt'+str(round(repair.processtime,2))+'_st:'+ str(round(repair.starttime,2))+'_cp:'+str(round(repair.completiontime,2))
            RepairsList.append(rep_str)   
           
                    
                    

        df = pd.DataFrame(columns=reportcolumns)

        df[df.columns[0]] = PNSNList

        colindex = 1
        stcol = 0
        compcol = 0
        proccol = 0
        for oprcomb in PO.designproduct.route:
            for operation in oprcomb.operations:
                df[df.columns[colindex]] = DFOpProcList[proccol]
                colindex+=1
                proccol+=1
                df[df.columns[colindex]] = DFOpStList[stcol]
                colindex+=1
                stcol+=1
                df[df.columns[colindex]] = DFOpCmpList[compcol]
                colindex+=1
                compcol+=1
        df[df.columns[-1]] = RepairsList

        return df
    
    

#***************************** P R O D U C T I O N       E X E C U T I O N ************************************
    
def SystemClock(env,ProdMgr,SupplyMngr,execdata,display,stateinfo,createlog):
   
    while True:
     
        for wcnt in ProdMgr.prodsystem.workcenters:   
            
            if wcnt.currentShift == None:
                ProdMgr.StartShift(wcnt,env.now,SupplyMngr,stateinfo)
            else: 
               
                for optr in wcnt.operators:  
                    
                    loginfo = optr.proceed(env,execdata,ProdMgr.prodsystem,wcnt.currentShift,display,createlog) 
                   
                    
                if env.now >= wcnt.currentShift.end: 
                    if wcnt.currentShift != None:
                        ProdMgr.CompleteShift(wcnt,stateinfo)
 
        unit_time = 0.01 # this is one minute 
        
        yield env.timeout(unit_time)
        
        
    return 

      

#********************************************************************************************#        
env = simpy.Environment()

display = False
generatedata = False
statewriting = False
checklog = False
st = time.time() # get the start time


planninghorizon = 5 # days
weekno = 21
d = "2023-W"+str(weekno)
weekstart = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")
print(weekstart)


completiontime = 1440*planninghorizon   # Sim time in minutes


#df_prodexedata = ReadProdData()


print('***************************** P R O D U C T I O N       S Y S T E M ************************************')
AME_System = ProductionSystem('AME')

ProdMgr = ProductionManager('AME_Manufacturing',AME_System)

ProdMgr.InitializeWC(AME_System)
#ProdMgr.InitializeWCO_OPT(df_prodexedata,AME_System)
ProdMgr.ReadResources('Resources.csv')

AME_System.report()

print('***************************** P R O D U C T I O N       S Y S T E M ************************************')

print('***************************** P R O D U C T I O N       D E S I G N ************************************')

DesignMgr = ProductDesignManager('AME_Design',AME_System)
DesignMgr.ReadOperationsProducts('ProductOperations.csv','Products.csv')

for PN,prod in DesignMgr.designproducts.items():
    prod.getBOM(0)  
print('***************************** P R O D U C T I O N       D E S I G N ************************************')


print('***************************** S U P P L Y   C H A I N ************************************')


print('Planning Horizon: ',planninghorizon,' days')
ProdMgr.createShifts(weekstart,'SA',planninghorizon)

SupplyMgr = SupplyChainManager('AME_SupplyChainManager')
SupplyMgr.ReadProductionOrders(DesignMgr,ProdMgr,weekstart)

print('Boxes created: ',sum([len(wc.allboxes) for wc in AME_System.workcenters]),' Trays created: ',sum([len(wc.alltrays) for wc in AME_System.workcenters]))
print('***************************** S U P P L Y   C H A I N ************************************')

print('***************************** P R O D U C T I O N       P L A N N I N G************************************')
ProdMgr.reportPlan()
print('***************************** P R O D U C T I O N       E X E C U T I O N ************************************')

env.process(SystemClock(env,ProdMgr,SupplyMgr,generatedata,display,statewriting,checklog))
            
# Execute
env.run(until = completiontime)
print('-> Execution time: ',  round(time.time() - st,2), 'seconds')



2023-05-22 00:00:00
***************************** P R O D U C T I O N       S Y S T E M ************************************
Defining a workcenter name SA
Workcenter SA  has  10  workstations
Availability info:  1901
name:  1771  comptence  []
Operator:  1771
Availability info size: 7
Competences: ['140 FT']
Availability: {datetime.date(2023, 3, 1): (27344.0, 55143.0), datetime.date(2023, 3, 2): (27181.0, 55202.0), datetime.date(2023, 3, 3): (27362.0, 55257.0)}
name:  1802  comptence  []
Operator:  1802
Availability info size: 190
Competences: ['140 FT', '150 SA']
Availability: {datetime.date(2023, 3, 1): (59990.0, 83378.0), datetime.date(2023, 3, 2): (56201.0, 82936.0), datetime.date(2023, 3, 3): (56041.0, 83453.0), datetime.date(2023, 3, 17): (59932.0, 81693.0), datetime.date(2023, 3, 20): (56585.0, 84605.0), datetime.date(2023, 3, 21): (56202.0, 83532.0), datetime.date(2023, 4, 5): (56082.0, 84886.0), datetime.date(2023, 4, 6): (55631.0, 84618.0), datetime.date(2023, 4, 7): (56394.0

Availability info size: 10
Competences: ['150 SA']
Availability: {datetime.date(2023, 5, 23): (68784.0, 84275.0)}
[140 FT]
[140 FT]
Workcnt:  SA  has 10 workstations, 2 testequipments, 23 operators
***************************** P R O D U C T I O N       S Y S T E M ************************************
***************************** P R O D U C T I O N       D E S I G N ************************************
   Unnamed: 0  Index              PN  Mean_ST  StDev_ST  Mean_PT  StDev_PT  \
0           0      0  6761-2200-1004        0         0      0.5         0   
1           1      1  6761-2200-1004        0         0      4.0         0   

  Description  BatchSize   Test  Automated  Parallel  SuccessRate  
0      150 SA          1  False      False     False            1  
1      140 FT          1   True       True      True            1  
0
1
Product 6761-2200-1004  comb ['150 SA'] op seq:  ['150 SA', '140 FT']
Product 6761-2200-1004  comb ['140 FT'] op seq:  ['150 SA', '140 FT']
***** Pro

-> 533.0 WS change (manual): Tray 9  -> box  6  in  SA_WS_3_Out_Buffer
-> 541.0 WS change (manual): Tray 10  -> box  6  in  SA_WS_3_Out_Buffer
 -->>> Transport (Optr  1907 ): boxes  [6] ,  SA_WS_3_Out_Buffer  >>>>  SA_WS_2_In_Buffer
    >> Transport Trays  [[6, 7, 8, 9, 10]]
 -->>> Transport (Optr  1903 ): boxes  [23, 24] ,  Buffer_Start  >>>>  SA_WS_3_In_Buffer
    >> Transport Trays  [[11, 12, 13, 14, 15], [16, 17, 18, 19, 20]]
CheckTray:  1907  has  [6, 7, 8, 9]  trays
CheckTray:  1903  has  [11, 12, 13, 14]  trays
-> 546.0 Production complete (unloading): Tray 6  -> box  139  in  SA_WS_2_Out_Buffer
-> 549.0 WS change (manual): Tray 11  -> box  138  in  SA_WS_3_Out_Buffer
-> 551.0 Production complete (unloading): Tray 7  -> box  139  in  SA_WS_2_Out_Buffer
-> 556.0 Production complete (unloading): Tray 8  -> box  139  in  SA_WS_2_Out_Buffer
-> 557.0 WS change (manual): Tray 12  -> box  138  in  SA_WS_3_Out_Buffer
-> 561.0 Production complete (unloading): Tray 9  -> box  139  in  SA_

-> 734.0 Production complete (unloading): Tray 41  -> box  153  in  SA_WS_2_Out_Buffer
-> 734.0 WS change (manual): Tray 46  -> box  154  in  SA_WS_1_Out_Buffer
-> 736.0 WS change (manual): Tray 54  -> box  151  in  SA_WS_3_Out_Buffer
CheckTray:  1903  has  [55, 56, 57, 58]  trays
-> 739.0 Production complete (unloading): Tray 42  -> box  153  in  SA_WS_2_Out_Buffer
-> 742.0 WS change (manual): Tray 47  -> box  154  in  SA_WS_1_Out_Buffer
-> 744.0 WS change (manual): Tray 55  -> box  151  in  SA_WS_3_Out_Buffer
 -->>> Transport (Optr  1903 ): boxes  [147, 151] ,  SA_WS_3_Out_Buffer  >>>>  SA_WS_2_In_Buffer
    >> Transport Trays  [[36, 37, 38, 39, 40], [51, 52, 53, 54, 55]]
-> 744.0 Production complete (unloading): Tray 43  -> box  153  in  SA_WS_2_Out_Buffer
-> 749.0 WS change (manual): Tray 48  -> box  154  in  SA_WS_1_Out_Buffer
-> 749.0 Production complete (unloading): Tray 44  -> box  153  in  SA_WS_2_Out_Buffer
CheckTray:  1896  has  [49, 50]  trays
CheckTray:  1907  has  [45, 36

Out_q buffer  SA_WS_10_Out_Buffer  items:  0
Bufferend  Buffer_End  items:  700
********** System state information ***********
Next opt:  150 SA  items:  4888
All trays of  150 SA  are placed!!
Next opt:  140 FT  items:  260
All trays of  140 FT  are placed!!
>>>> Configuring Workcenter  SA  in shift  2023-05-22 Night , [ 930 , 1439 ]:
No state reading...
Products:  5848 Produced:  700
Boxes:  173 Trays:  585
Time:  930.0099999992992  shift start:   930  released:  True
 ---> Operation  ['150 SA']  @Workstation:   SA_WS_1  Optr:  2003 , Availability: [ 955 1383 ]
  >> Test equipment  SA_TestMach1 -> Workstation  SA_WS_2
  >> Test equipment  SA_TestMach2 -> Workstation  SA_WS_2
 ---> Operation  ['140 FT']  @Workstation:   SA_WS_2  Optr:  2025 , Availability: [ 940 1398 ]
Production targets, shift.POs:  1 , new POs to start:  0
Available operators [2003, 2025]  in shift:  2023-05-22 Night
shift set:  930  POs to produce:  1
[(43, 50), (44, 50), (45, 50), (46, 50), (47, 50), (48, 50), (4

...

-> 4875.0 WS change (manual): Tray 522  -> box  406  in  SA_WS_1_Out_Buffer
-> 4875.0 Production complete (unloading): Tray 453  -> box  426  in  SA_WS_2_Out_Buffer
-> 4877.0 WS change (manual): Tray 520  -> box  411  in  SA_WS_3_Out_Buffer
 -->>> Transport (Optr  1929 ): boxes  [410, 411] ,  SA_WS_3_Out_Buffer  >>>>  SA_WS_2_In_Buffer
    >> Transport Trays  [[511, 512, 513, 514, 515], [516, 517, 518, 519, 520]]
 -->>> Transport (Optr  1929 ): boxes  [127, 128] ,  Buffer_Start  >>>>  SA_WS_3_In_Buffer
    >> Transport Trays  [[531, 532, 533, 534, 535], [536, 537, 538, 539, 540]]
CheckTray:  1929  has  [531, 532, 533, 534]  trays
-> 4882.0 Production complete (unloading): Tray 454  -> box  426  in  SA_WS_2_Out_Buffer
CheckTray:  1907  has  [455, 456, 457, 458]  trays
-> 4883.0 WS change (manual): Tray 523  -> box  406  in  SA_WS_1_Out_Buffer
[------ Operator 1896  starts ----- B R E A K  4884 - 5011 -----]
-> 4887.0 WS change (manual): Tray 531  -> box  428  in  SA_WS_3_Out_Buffer
-> 4

-> 5120.0 Production complete (unloading): Tray 526  -> box  439  in  SA_WS_2_Out_Buffer
-> 5120.0 WS change (manual): Tray 564  -> box  440  in  SA_WS_3_Out_Buffer
CheckTray:  1929  has  [565, 566, 567, 568]  trays
-> 5125.0 WS change (manual): Tray 557  -> box  437  in  SA_WS_1_Out_Buffer
-> 5127.0 Production complete (unloading): Tray 527  -> box  439  in  SA_WS_2_Out_Buffer
-> 5130.0 WS change (manual): Tray 565  -> box  440  in  SA_WS_3_Out_Buffer
-> 5133.0 WS change (manual): Tray 558  -> box  437  in  SA_WS_1_Out_Buffer
CheckTray:  1896  has  [559, 560]  trays
-> 5134.0 Production complete (unloading): Tray 528  -> box  439  in  SA_WS_2_Out_Buffer
CheckTray:  1907  has  [529, 530, 541, 542]  trays
-> 5141.0 WS change (manual): Tray 566  -> box  441  in  SA_WS_3_Out_Buffer
-> 5141.0 WS change (manual): Tray 559  -> box  437  in  SA_WS_1_Out_Buffer
-> 5142.0 Production complete (unloading): Tray 529  -> box  439  in  SA_WS_2_Out_Buffer
-> 5149.0 WS change (manual): Tray 560  -> bo

-> 5310.0 Production complete (unloading): Tray 475  -> box  452  in  SA_WS_2_Out_Buffer
-> 5315.0 Production complete (unloading): Tray 501  -> box  453  in  SA_WS_2_Out_Buffer
-> 5320.0 Production complete (unloading): Tray 502  -> box  453  in  SA_WS_2_Out_Buffer
-> 5325.0 Production complete (unloading): Tray 503  -> box  453  in  SA_WS_2_Out_Buffer
CheckTray:  1802  has  [504, 505]  trays
-> 5330.0 Production complete (unloading): Tray 504  -> box  453  in  SA_WS_2_Out_Buffer
-> 5335.0 Production complete (unloading): Tray 505  -> box  453  in  SA_WS_2_Out_Buffer
 -->>> Transport (Optr  1802 ): boxes  [452, 453] ,  SA_WS_2_Out_Buffer  >>>>  Buffer_End
    >> Transport Trays  [[471, 472, 473, 474, 475], [501, 502, 503, 504, 505]]
 -->>> Transport (Optr  1802 ): boxes  [348, 349] ,  Buffer_Start  >>>>  SA_WS_2_In_Buffer
    >> Transport Trays  [[506, 507, 498, 499, 500], [571, 572, 573, 574, 575]]
CheckTray:  1802  has  [506, 507, 498, 499]  trays
-> 5340.0 Production complete (unlo

-> 5590.0 WS change (manual): Tray 467  -> box  484  in  SA_WS_1_Out_Buffer
-> 5591.0 Production complete (unloading): Tray 297  -> box  483  in  SA_WS_2_Out_Buffer
 -->>> Transport (Optr  1802 ): boxes  [482, 483] ,  SA_WS_2_Out_Buffer  >>>>  Buffer_End
    >> Transport Trays  [[177, 178, 179, 180, 292], [293, 294, 295, 296, 297]]
-> 5596.0 Production complete (unloading): Tray 298  -> box  486  in  SA_WS_2_Out_Buffer
-> 5597.0 WS change (manual): Tray 468  -> box  484  in  SA_WS_1_Out_Buffer
-> 5601.0 Production complete (unloading): Tray 299  -> box  486  in  SA_WS_2_Out_Buffer
CheckTray:  1802  has  [300, 280, 365, 366]  trays
-> 5604.0 WS change (manual): Tray 469  -> box  484  in  SA_WS_1_Out_Buffer
CheckTray:  2003  has  [470, 476, 477, 478]  trays
-> 5607.0 Production complete (unloading): Tray 300  -> box  486  in  SA_WS_2_Out_Buffer
-> 5611.0 WS change (manual): Tray 470  -> box  484  in  SA_WS_1_Out_Buffer
-> 5612.0 Production complete (unloading): Tray 280  -> box  486  in 

-> 6285.0 Production complete (unloading): Tray 107  -> box  498  in  SA_WS_2_Out_Buffer
 -->>> Transport (Optr  1907 ): boxes  [497, 498] ,  SA_WS_2_Out_Buffer  >>>>  Buffer_End
    >> Transport Trays  [[580, 508, 509, 510, 578], [579, 110, 96, 106, 107]]
CheckTray:  1907  has  [108]  trays
-> 6290.0 Production complete (unloading): Tray 108  -> box  516  in  SA_WS_2_Out_Buffer
 -->>> Transport (Optr  1907 ): boxes  [496, 515] ,  SA_WS_1_Out_Buffer  >>>>  SA_WS_2_In_Buffer
    >> Transport Trays  [[581, 582, 583, 584, 585], [109]]
CheckTray:  1907  has  [581, 582, 583, 584]  trays
-> 6295.0 Production complete (unloading): Tray 581  -> box  516  in  SA_WS_2_Out_Buffer
-> 6300.0 Production complete (unloading): Tray 582  -> box  516  in  SA_WS_2_Out_Buffer
-> 6305.0 Production complete (unloading): Tray 583  -> box  516  in  SA_WS_2_Out_Buffer
-> 6311.0 Production complete (unloading): Tray 584  -> box  516  in  SA_WS_2_Out_Buffer
CheckTray:  1907  has  [585, 109]  trays
-> 6316.0 Prod

In [38]:
import pandas as pd
import datetime
from datetime import timedelta,date


df = pd.DataFrame(columns=['Workcenterid','Name','Abbreviation','No.Workstations'])

newwc = {'Workcenterid':1,'Name':'System Assembly','Abbreviation':'SA','No.Workstations':5}

df = df.append(newwc, ignore_index=True)

df.to_csv('simdata/Workcenters.csv')


#df.head(5)

df2 = pd.DataFrame(columns=['Workcenter','PN','ItemsonTray','TraysinBox'])

newprod = {'Workcenter':'SA','PN':'6761-2200-1004','ItemsonTray':10,'TraysinBox':5}

df2 = df2.append(newprod, ignore_index=True)

df2.to_csv('simdata/Products.csv')


df3 = pd.DataFrame(columns=['Index','PN','Mean_ST','StDev_ST','Mean_PT','StDev_PT','Description','BatchSize','Test','Automated','Parallel','SuccessRate'])

newop1 = {'Index':0,'PN':'6761-2200-1004','Mean_ST':0,'StDev_ST':0,'Mean_PT':0.55,'StDev_PT':0,'Description':'150 SA','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
newop2 = {'Index':1,'PN':'6761-2200-1004','Mean_ST':0.,'StDev_ST':0,'Mean_PT':4,'StDev_PT':0,'Description':'140 FT','BatchSize':1,'Test':True,'Automated':True,'Parallel':True,'SuccessRate':1.0}

#newop1 = {'Index':0,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Spring','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop2 = {'Index':1,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Solonoid','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop3 = {'Index':2,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Press','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop4 = {'Index':3,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'PCB','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop5 = {'Index':4,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Label','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop6 = {'Index':5,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Place top cover','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop7 = {'Index':6,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Screw','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop8 = {'Index':7,'PN':'6761-1903-0303','Mean_ST':0.1,'StDev_ST':0.01,'Mean_PT':0.2,'StDev_PT':0.5,'Description':'Sticker','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop9 = {'Index':1,'PN':'6761-1903-0303','Mean_ST':0,'StDev_ST':0,'Mean_PT':3.2,'StDev_PT':0.01,'Description':'140 FT','BatchSize':4,'Test':True,'Automated':True,'Parallel':True,'SuccessRate':1.00}
#newop10 = {'Index':9,'PN':'6761-1903-0303','Mean_ST':0.5,'StDev_ST':0.01,'Mean_PT':1,'StDev_PT':0.1,'Description':'Boxing','BatchSize':40,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}
#newop11 = {'Index':-1,'PN':'6761-1903-0303','Mean_ST':0.5,'StDev_ST':0.01,'Mean_PT':4,'StDev_PT':1,'Description':'Repair_op','BatchSize':1,'Test':False,'Automated':False,'Parallel':False,'SuccessRate':1.0}

'''
df3 = df3.append(newop1, ignore_index=True)
df3 = df3.append(newop2, ignore_index=True)
df3 = df3.append(newop3, ignore_index=True)
df3 = df3.append(newop4, ignore_index=True)
df3 = df3.append(newop5, ignore_index=True)
df3 = df3.append(newop6, ignore_index=True)
df3 = df3.append(newop7, ignore_index=True)
df3 = df3.append(newop8, ignore_index=True)
'''
df3 = df3.append(newop1, ignore_index=True)
df3 = df3.append(newop2, ignore_index=True)
#df3 = df3.append(newop9, ignore_index=True)
'''
df3 = df3.append(newop10, ignore_index=True)
df3 = df3.append(newop11, ignore_index=True)
'''


df3.to_csv('simdata/ProductOperations.csv')

df3a = pd.DataFrame(columns=['Index','PN','Description','Operations'])

newopc1 = {'Index':0,'PN':'6761-2200-1004','Description':'Comb1','Operations':'[150 SA]'}
newopc2 = {'Index':1,'PN':'6761-2200-1004','Description':'Comb2','Operations':'[140 FT]'}
df3a = df3a.append(newopc1, ignore_index=True)
df3a = df3a.append(newopc2, ignore_index=True)
df3a.to_csv('simdata/OperationCombinations.csv')

df4 = pd.DataFrame(columns=['ResourceType','Name','ItemsPerRun','Utilization','Competence','Workcenter','RecoverTime'])

#ProdMgr.defineOperator('SA_Opt1','SA',['SA_op'])
#ProdMgr.defineOperator('SA_Opt2','SA',['FT_op'])

newres = {'ResourceType':'Human','Name':'Opt_1907','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newres1 = {'ResourceType':'Human','Name':'Opt_1771','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newres2 = {'ResourceType':'Human','Name':'Opt_1896','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}


newresx = {'ResourceType':'Human','Name':'Opt_1802','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newresy = {'ResourceType':'Human','Name':'Opt_1860','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticke]','Workcenter':'SA','RecoverTime':0}
newresz = {'ResourceType':'Human','Name':'Opt_2025','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newrest = {'ResourceType':'Human','Name':'Opt_1818','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newreszz = {'ResourceType':'Human','Name':'Opt_1940','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newreszzz = {'ResourceType':'Human','Name':'Opt_1241','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}
newreszzxz = {'ResourceType':'Human','Name':'Opt_1678','ItemsPerRun':1,'Utilization':1,'Competence':'[Spring,Solonoid,Press,PCB,Label,Place top cover,Screw,Sticker]','Workcenter':'SA','RecoverTime':0}



newres3 = {'ResourceType':'Human','Name':'Opt_4','ItemsPerRun':1,'Utilization':1,'Competence':'[Repair_op]','Workcenter':'SA','RecoverTime':0}
newres4 = {'ResourceType':'TestEquipment','Name':'SA_TestMach1','ItemsPerRun':5,'Utilization':0.7,'Competence':'[140 FT]','Workcenter':'SA','RecoverTime':180}
newres5 = {'ResourceType':'TestEquipment','Name':'SA_TestMach2','ItemsPerRun':5,'Utilization':0.7,'Competence':'[140 FT]','Workcenter':'SA','RecoverTime':180}

df4 = df4.append(newres, ignore_index=True)
df4 = df4.append(newres1, ignore_index=True)
df4 = df4.append(newres2, ignore_index=True)
df4 = df4.append(newres3, ignore_index=True)
df4 = df4.append(newres4, ignore_index=True)
df4 = df4.append(newres5, ignore_index=True)
df4 = df4.append(newresx, ignore_index=True)
df4 = df4.append(newresy, ignore_index=True)
df4 = df4.append(newresz, ignore_index=True)
df4 = df4.append(newrest, ignore_index=True)
df4 = df4.append(newreszz, ignore_index=True)
df4 = df4.append(newreszzz, ignore_index=True)
df4 = df4.append(newreszzxz, ignore_index=True)
df4.to_csv('simdata/Resources.csv')


df_availability = pd.DataFrame(columns=['TransactionUser_ID','Day','ActivePeriodStart','ActivePeriodEnd'])


today = datetime.date.today()


day = date(today.year,today.month,today.day)

userids = [1907,2025,1897]
starts = [20000,30000,40000]
lengths=  [20000,30000,25000]

for useridx in range(len(userids)): 
    
    crday = date(today.year,today.month,today.day)

    newav = {'TransactionUser_ID':userids[useridx],'Day':crday,'ActivePeriodStart': starts[useridx],'ActivePeriodEnd':starts[useridx]+lengths[useridx]}
    df_availability = df_availability.append(newav, ignore_index=True)
    crday = crday + timedelta(days =1)
    newav = {'TransactionUser_ID':userids[useridx],'Day':crday,'ActivePeriodStart': starts[useridx],'ActivePeriodEnd':starts[useridx]+lengths[useridx]}
    df_availability = df_availability.append(newav, ignore_index=True)
    
    
#df_availability.to_csv('simdata/Availability.csv')







### State file generation template

In [8]:

AME_System = ProductionSystem('AME')
ProdMgr = ProductionManager('AME_Manufacturing',AME_System)

ProdMgr.InitializeWorkcenters('Workcenters.csv',AME_System)
ProdMgr.ReadResources('Resources.csv')

DesignMgr = ProductDesignManager('AME_Design',AME_System)
DesignMgr.ReadOperationsProducts('ProductOperations.csv','Products.csv')

print('Planning Horizon: ',planninghorizon,' days')
ProdMgr.createShifts(weekstart,'SA',planninghorizon)

SupplyMgr = SupplyChainManager('AME_SupplyChainManager')
SupplyMgr.ReadProductionOrders(DesignMgr,ProdMgr,weekstart)

PO = SupplyMgr.POs[0]

reportcolumns=['Day']
 
    
DayList = []
DFBuffList = []
    
for oprcomb in PO.designproduct.route:
    for operation in oprcomb.operations:
        DFBuffList.append([])
        reportcolumns.append(operation.name+"_buffer")
       
    
df_states = pd.DataFrame(columns=reportcolumns)


today = datetime.date.today()


day = date(today.year,today.month,today.day)

currentday  = day +  timedelta(days=-20)

while currentday < day:
    
    
    DayList.append(currentday)
    currentday  = currentday +  timedelta(days = 1)
    
    for currcol in range(1,len(reportcolumns)):
        DFBuffList[currcol-1].append(200)
    

df_states[df_states.columns[0]] = DayList

colindex = 1
        
for currcol in range(1,len(reportcolumns)):
    df_states[df_states.columns[currcol]] = DFBuffList[currcol-1]

             
df_states.to_csv('simdata/SystemState_'+PO.designproduct.PN+'.csv')   


Defining a workcenter name SA
Availability info:  457
name:  Opt_1907  comptence  ['Spring', 'Solonoid', 'Press', 'PCB', 'Label', 'Place top cover', 'Screw', 'Sticker', '150 SA']
Operator 1907  has availability info:  69
name:  Opt_1771  comptence  ['Spring', 'Solonoid', 'Press', 'PCB', 'Label', 'Place top cover', 'Screw', 'Sticker', '140 FT', 'Boxing']
Operator 1771  has availability info:  20
name:  Opt_1896  comptence  ['Spring', 'Solonoid', 'Press', 'PCB', 'Label', 'Place top cover', 'Screw', 'Sticker', '150 SA']
Operator 1896  has availability info:  50
name:  Opt_4  comptence  ['Repair_op']
Operator 4  has availability info:  0
name:  Opt_1802  comptence  ['Spring', 'Solonoid', 'Press', 'PCB', 'Label', 'Place top cover', 'Screw', 'Sticker', '140 FT', '150 SA', 'Boxing']
Operator 1802  has availability info:  35
name:  Opt_1860  comptence  ['Spring', 'Solonoid', 'Press', 'PCB', 'Label', 'Place top cover', 'Screw', 'Sticker', '150 SA', '140 FT']
Operator 1860  has availability info

In [23]:
import pandas as pd
import datetime

weekno = 20

d = "2023-W"+str(weekno)
rts = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")



df_po = pd.DataFrame(columns=['PO','PN','Quantity','Date','ParallelExecutions'])

newpo = {'PO':1,'PN':'6761-2200-1004','Quantity':1000,'Date':rts,'ParallelExecutions':'[1,1]'}

df_po = df_po.append(newpo, ignore_index=True)

df_po.to_csv('simdata/ProductionOrders.csv')


df_POs = pd.read_csv('simdata/ProductionOrders.csv')

df_POs['Date']= pd.to_datetime(df_POs['Date'])
        
for x,r in df_POs.iterrows():
            
    time = (r['Date'] -rts).total_seconds() / 60
    if time%1440 < 420:
        time = int(time/1440)*1440+420
            
    print(time)
            

420


In [3]:
import numpy as np 
import pandas as pd

opsfile = 'ProductOperations.csv'
df_ops = pd.read_csv('simdata/'+opsfile)
df_ops = df_ops.convert_dtypes()
df_ops.head(5)


,Unnamed: 0,Index,PN,Mean_ST,StDev_ST,Mean_PT,StDev_PT,Description,BatchSize,Test,Automated,Parallel,SuccessRate
0,0,0,6761-1903-0303,0.1,0.01,0.2,0.5,Spring,1,False,False,False,1.0
1,1,1,6761-1903-0303,0.1,0.01,0.2,0.5,Solonoid,1,False,False,False,1.0
2,2,2,6761-1903-0303,0.1,0.01,0.2,0.5,Press,1,False,False,False,1.0
3,3,3,6761-1903-0303,0.1,0.01,0.2,0.5,PCB,1,False,False,False,1.0
4,4,4,6761-1903-0303,0.1,0.01,0.2,0.5,Label,1,False,False,False,1.0


In [10]:
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)



2023-05-15 00:00:00


In [7]:
opsfile = 'Shift_480_CapacityUse.csv'
df = pd.read_csv('simdata/'+opsfile)
df = df.convert_dtypes()
df.head(5)


,Unnamed: 0,Buffer,Time,CapacityUse
0,0,SA_RepairStation_In_Buffer,0.0,0
1,1,SA_WS_1_In_Buffer,0.0,0
2,2,SA_WS_1_In_Buffer,481.2,1
3,3,SA_WS_1_In_Buffer,481.2,1
4,4,SA_WS_1_In_Buffer,481.2,1
